In [1]:
# This is the example code for TA creation from CRM and Sales data
'''
1. Fetch historical sales by id and merge in zipcode, then roll up to sales/trans (or others) by store by zip and rank
2. For each store, assign the zips which contributed top 60% (P) and the next 20% (S). The rest 20% T can be everywhere =.=
3. Once defined the zips by each store, combine stores into TA based on the overlap
4. Summarize to output
'''

'\n1. Fetch historical sales by id and merge in zipcode, then roll up to sales/trans (or others) by store by zip and rank\n2. For each store, assign the zips which contributed top 60% (P) and the next 20% (S). The rest 20% T can be everywhere =.=\n3. Once defined the zips by each store, combine stores into TA based on the overlap\n4. Summarize to output\n'

In [ ]:
TA-Store list
Store-zips list

TA -- Store -- zips

In [2]:
# Load packages and access

import pandas as pd
import os
import datetime
import numpy as np
from haversine import haversine
from google.cloud import bigquery
import glob
import json
import gc
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/docs/merkle-gdyr-prod-9783607c77a1.json"

pd.options.display.max_columns=70
zip_centers=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
path_nielsen_zip="/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx"
num_miles_to_create_TA=40
num_final_inclusion_dist=40

list_str_stores_to_remove=['1280','1288','3605']

print(os.getcwd())
print(datetime.datetime.now())

client=bigquery.Client()

/mnt/clients/juba/hqjubaapp02/sharefolder/TraderArea/GoodYear_TA_example_CRM
2020-12-01 13:59:29.261206


In [3]:
# Pull sales

query_string='''
with t_mapping_id_zip as (
select t1.indiv_key, LPAD(ifnull(t2.postal_cd,"00000"), 5, '0') as zip_5, from mdb.indiv_summary as t1 
left join mdb.addr as t2 on t1.cr_addr_key=t2.cr_addr_key
)

select store_id, ifnull(zip_5,'99999') as zip_5, count(distinct trn_id) as unique_trans, sum(line_item_total_amt) as total_sales from mdb.trn_detail as t1
left join t_mapping_id_zip as t2
on t1.indiv_key=t2.indiv_key
where invoice_date between DATE_SUB(CURRENT_DATE(), interval 3 YEAR) and DATE_SUB(CURRENT_DATE(), interval 1 DAY)
group by 1, 2
order by 1, 3 desc,4 desc;
'''
print("query: \n%s"%query_string)
df_store_sales_by_zip=client.query(query_string).result().to_dataframe()
print("df_store_sales_by_zip.shape",df_store_sales_by_zip.shape)

query: 

with t_mapping_id_zip as (
select t1.indiv_key, LPAD(ifnull(t2.postal_cd,"00000"), 5, '0') as zip_5, from mdb.indiv_summary as t1 
left join mdb.addr as t2 on t1.cr_addr_key=t2.cr_addr_key
)

select store_id, ifnull(zip_5,'99999') as zip_5, count(distinct trn_id) as unique_trans, sum(line_item_total_amt) as total_sales from mdb.trn_detail as t1
left join t_mapping_id_zip as t2
on t1.indiv_key=t2.indiv_key
where invoice_date between DATE_SUB(CURRENT_DATE(), interval 3 YEAR) and DATE_SUB(CURRENT_DATE(), interval 1 DAY)
group by 1, 2
order by 1, 3 desc,4 desc;



/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


df_store_sales_by_zip.shape (433452, 4)


In [4]:
# Pull store list

query_string='''
select store_id, min(cast(invoice_date as date)) as min_date, max(cast(invoice_date as date)) as max_date, count(distinct cast(invoice_date as date)) as unique_dates from mdb.trn_detail 
where invoice_date between DATE_SUB(CURRENT_DATE(), interval 3 YEAR) and DATE_SUB(CURRENT_DATE(), interval 1 DAY)
group by store_id
order by store_id;
'''
print("query: \n%s"%query_string)
df_store_open_dates=client.query(query_string).result().to_dataframe()
print("df_store_open_dates.shape",df_store_open_dates.shape)


query: 

select store_id, min(cast(invoice_date as date)) as min_date, max(cast(invoice_date as date)) as max_date, count(distinct cast(invoice_date as date)) as unique_dates from mdb.trn_detail 
where invoice_date between DATE_SUB(CURRENT_DATE(), interval 3 YEAR) and DATE_SUB(CURRENT_DATE(), interval 1 DAY)
group by store_id
order by store_id;

df_store_open_dates.shape (593, 4)


In [5]:
# email from YG 2020-10-22 at 10:03 AM: Corporate stores with NO transactions in the last 12 months should considers closed
# Set some condition to define store open and whether valid to include in the computing
max_date=df_store_open_dates['max_date'].max()
date_1_year_ago=max_date-datetime.timedelta(days=365)
df_store_open_dates=df_store_open_dates[df_store_open_dates['max_date']>=date_1_year_ago]
df_store_open_dates=df_store_open_dates[df_store_open_dates['unique_dates']>90]
df_store_open_dates.shape

(574, 4)

In [6]:
# Pull open stores

# currently open and coorperated owned stores
query_string='''
SELECT * from mdb.vw__installer_master as t1
left join mdb_cdl.store t2 on t1.store_num=t2.store_num
where t1.company_owned_store_ind="Y" and open_store_flag='Y';
'''

print("query: \n%s"%query_string)
df_store_included=client.query(query_string).result().to_dataframe()
print("df_store_included.shape",df_store_included.shape)


query: 

SELECT * from mdb.vw__installer_master as t1
left join mdb_cdl.store t2 on t1.store_num=t2.store_num
where t1.company_owned_store_ind="Y" and open_store_flag='Y';

df_store_included.shape (573, 63)


In [7]:
# Just format the int and str

print(df_store_sales_by_zip['store_id'].apply(type).unique())
print(df_store_open_dates['store_id'].apply(type).unique())
print(df_store_included['store_num'].apply(type).unique())

df_store_sales_by_zip=df_store_sales_by_zip[df_store_sales_by_zip['store_id'].str.isdigit()]
df_store_open_dates=df_store_open_dates[df_store_open_dates['store_id'].str.isdigit()]
df_store_included=df_store_included[df_store_included['store_num'].str.isdigit()]

df_store_sales_by_zip['store_id']=df_store_sales_by_zip['store_id'].astype(int).astype(str)
df_store_open_dates['store_id']=df_store_open_dates['store_id'].astype(int).astype(str)
df_store_included['store_num']=df_store_included['store_num'].astype(int).astype(str)

print(df_store_sales_by_zip['store_id'].apply(type).unique())
print(df_store_open_dates['store_id'].apply(type).unique())
print(df_store_included['store_num'].apply(type).unique())

[<class 'str'>]
[<class 'str'>]
[<class 'str'>]
[<class 'str'>]
[<class 'str'>]
[<class 'str'>]


In [8]:
# Merge in and keeping only valid stores

df_store_included_with_dates=pd.merge(df_store_included,df_store_open_dates,left_on="store_num",right_on="store_id",how="inner")
# Remove 3 spread stores
df_store_included_with_dates=df_store_included_with_dates[~df_store_included_with_dates['store_num'].isin(list_str_stores_to_remove)]

df_store_included=df_store_included[df_store_included['store_num'].isin(df_store_included_with_dates['store_num'].tolist())]
df_store_included_with_dates.shape

(567, 67)

In [9]:
# Same here, keep valid stores only for the POS details
list_inclusion_stores=df_store_included_with_dates['store_num'].unique().tolist()
print(len(list_inclusion_stores))
df_store_sales_by_zip=df_store_sales_by_zip[df_store_sales_by_zip['store_id'].isin(list_inclusion_stores)]
print(df_store_sales_by_zip.shape)
df_store_sales_by_zip['store_id'].nunique()

567
(404118, 4)


567

In [17]:
df_store_sales_by_zip.head(3)

,store_id,zip_5,unique_trans,total_sales,store_trans,store_sales,trans_cum,trans_rank,trans_cum_pctg,sales_cum,sales_rank,sales_cum_pctg,trans_label,sales_label,latitude,longitude
52938,1023,15601,6905,1467085.04,17597,4141636.76,6905,1,0.392396,1467085.04,1,0.354228,P,P,40.3048,-79.5831
52939,1023,15644,2207,461576.88,17597,4141636.76,9112,2,0.517816,1928661.92,2,0.465676,S,S,40.3048,-79.5831
52940,1023,15642,1501,396914.72,17597,4141636.76,10613,3,0.603114,2325576.64,3,0.561511,S,P,40.3048,-79.5831


In [10]:
# Rank and assign the P/S repesctively by transaction and sales. We use transaction here for this client


df_store_sales_total=df_store_sales_by_zip.groupby('store_id')['unique_trans','total_sales'].sum().reset_index()
df_store_sales_total=df_store_sales_total.rename(columns={"unique_trans":"store_trans","total_sales":"store_sales"})
df_store_sales_by_zip=pd.merge(df_store_sales_by_zip,df_store_sales_total,on="store_id")
df_store_sales_by_zip['temp_row_count']=1

df_store_sales_by_zip=df_store_sales_by_zip.sort_values(['store_id','unique_trans','total_sales'],ascending=[True,False,False])
df_store_sales_by_zip['trans_cum'] = df_store_sales_by_zip.groupby(['store_id'])['unique_trans'].cumsum()
df_store_sales_by_zip['trans_rank'] = df_store_sales_by_zip.groupby(['store_id'])['temp_row_count'].cumsum()
df_store_sales_by_zip['trans_cum_pctg']=df_store_sales_by_zip['trans_cum']/df_store_sales_by_zip['store_trans']

df_store_sales_by_zip=df_store_sales_by_zip.sort_values(['store_id','total_sales','unique_trans'],ascending=[True,False,False])
df_store_sales_by_zip['sales_cum'] = df_store_sales_by_zip.groupby(['store_id'])['total_sales'].cumsum()
df_store_sales_by_zip['sales_rank'] = df_store_sales_by_zip.groupby(['store_id'])['temp_row_count'].cumsum()
df_store_sales_by_zip['sales_cum_pctg']=df_store_sales_by_zip['sales_cum']/df_store_sales_by_zip['store_sales']

df_store_sales_by_zip['trans_label']=np.where(((df_store_sales_by_zip['trans_cum_pctg']<=0.6) | (df_store_sales_by_zip['trans_rank']==1)),"P",
                                             np.where(df_store_sales_by_zip['trans_cum_pctg']<=0.8,"S","T")
                                             )
df_store_sales_by_zip['sales_label']=np.where(((df_store_sales_by_zip['sales_cum_pctg']<=0.6) | (df_store_sales_by_zip['sales_rank']==1)),"P",
                                             np.where(df_store_sales_by_zip['sales_cum_pctg']<=0.8,"S","T")
                                             )
# To make sure to have at least an S
df_store_sales_by_zip['trans_label']=np.where(((df_store_sales_by_zip['trans_label']=="T") | (df_store_sales_by_zip['trans_rank']==2)),"S",
                                              df_store_sales_by_zip['trans_label']
                                             )
df_store_sales_by_zip['sales_label']=np.where(((df_store_sales_by_zip['sales_label']=="T") | (df_store_sales_by_zip['sales_rank']==2)),"S",
                                              df_store_sales_by_zip['sales_label']
                                             )


del df_store_sales_by_zip['temp_row_count']
df_store_sales_by_zip.head(2)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,store_id,zip_5,unique_trans,total_sales,store_trans,store_sales,trans_cum,trans_rank,trans_cum_pctg,sales_cum,sales_rank,sales_cum_pctg,trans_label,sales_label
52938,1023,15601,6905,1467085.04,17597,4141636.76,6905,1,0.392396,1467085.04,1,0.354228,P,P
52939,1023,15644,2207,461576.88,17597,4141636.76,9112,2,0.517816,1928661.92,2,0.465676,S,S


In [11]:
# QC all store have P&S

# In case that 1 zip contribute more than 80% transction, QC here and you can find that the top is assigned to P and then S in the previous cell
df=df_store_sales_by_zip.groupby("trans_label")['store_id'].nunique().reset_index()
df

,trans_label,store_id
0,P,567
1,S,567


In [12]:
# Ensured that we have at least 1 P zips, but may lack of S zips which is fine
list_S_store=df_store_sales_by_zip[df_store_sales_by_zip['trans_label']=="S"]['store_id'].unique().tolist()
df=df_store_sales_by_zip[~df_store_sales_by_zip['store_id'].isin(list_S_store)]

for store,df_group in df.groupby('store_id'):
    df_group=df_group[df_group['trans_label']!="P"]
    print(store,df_group['trans_cum_pctg'].min())
    print(df_group['trans_cum_pctg'].tolist()[:3])

In [13]:
# Add the lat/lng to the sales DF

dict_store_lat=df_store_included.set_index("store_num").to_dict()['latitude']
dict_store_lng=df_store_included.set_index("store_num").to_dict()['longitude']

df_store_sales_by_zip['latitude']=df_store_sales_by_zip['store_id'].apply(lambda x: dict_store_lat[x])
df_store_sales_by_zip['longitude']=df_store_sales_by_zip['store_id'].apply(lambda x: dict_store_lng[x])
df_store_sales_by_zip.head(2)

,store_id,zip_5,unique_trans,total_sales,store_trans,store_sales,trans_cum,trans_rank,trans_cum_pctg,sales_cum,sales_rank,sales_cum_pctg,trans_label,sales_label,latitude,longitude
52938,1023,15601,6905,1467085.04,17597,4141636.76,6905,1,0.392396,1467085.04,1,0.354228,P,P,40.3048,-79.5831
52939,1023,15644,2207,461576.88,17597,4141636.76,9112,2,0.517816,1928661.92,2,0.465676,S,S,40.3048,-79.5831


In [14]:
# QC zips that we don't have thier lat/lng (client's data can contain whatever =.=)
df=df_store_sales_by_zip[~df_store_sales_by_zip['zip_5'].isin(list(zip_centers.keys()))]
df.groupby("trans_label")['store_id'].count().to_frame()

,store_id
trans_label,
P,11
S,5145


In [15]:
# Add the distence between store and zip

df_sales_by_zip=df_store_sales_by_zip.copy()
df_sales_by_zip=df_sales_by_zip[df_sales_by_zip['zip_5'].isin(list(zip_centers.keys()))]

df_sales_by_zip=df_sales_by_zip.reset_index()
del df_sales_by_zip['index']

def cal_dist_zip_store_miles(zip_cd,store_lat,store_lng):
    return haversine(zip_centers[zip_cd],(store_lat,store_lng),unit="mi")

df_sales_by_zip['dist_miles']=df_sales_by_zip.apply(lambda df: cal_dist_zip_store_miles(df['zip_5'],df['latitude'],df['longitude']),axis=1)


df_sales_by_zip.head(5)

,store_id,zip_5,unique_trans,total_sales,store_trans,store_sales,trans_cum,trans_rank,trans_cum_pctg,sales_cum,sales_rank,sales_cum_pctg,trans_label,sales_label,latitude,longitude,dist_miles
0,1023,15601,6905,1467085.04,17597,4141636.76,6905,1,0.392396,1467085.04,1,0.354228,P,P,40.3048,-79.5831,2.283179
1,1023,15644,2207,461576.88,17597,4141636.76,9112,2,0.517816,1928661.92,2,0.465676,S,S,40.3048,-79.5831,2.576020
2,1023,15642,1501,396914.72,17597,4141636.76,10613,3,0.603114,2325576.64,3,0.561511,S,P,40.3048,-79.5831,7.150137
3,1023,15650,531,126187.85,17597,4141636.76,11144,4,0.633290,2451764.49,4,0.591980,S,P,40.3048,-79.5831,9.946670
4,1023,15231,115,109013.23,17597,4141636.76,13233,16,0.752003,2560777.72,5,0.618301,S,S,40.3048,-79.5831,38.556427


In [16]:
# Write into csv as tempory copy
df_store_sales_by_zip.to_csv('./df_store_sales_by_zip.csv',index=False)
df_sales_by_zip.to_csv('./df_sales_by_zip.csv',index=False)
gc.collect()

20

# Creating TA

In [19]:
# Create a folder for the new outputs

output_folder="./output_%s/"%str(datetime.datetime.now().date())
try:
    os.stat(output_folder)
except:
    os.mkdir(output_folder)
    


In [20]:
# Take a look at the dataframe above
df_store_sales_by_zip.head(2)

,store_id,zip_5,unique_trans,total_sales,store_trans,store_sales,trans_cum,trans_rank,trans_cum_pctg,sales_cum,sales_rank,sales_cum_pctg,trans_label,sales_label,latitude,longitude
52938,1023,15601,6905,1467085.04,17597,4141636.76,6905,1,0.392396,1467085.04,1,0.354228,P,P,40.3048,-79.5831
52939,1023,15644,2207,461576.88,17597,4141636.76,9112,2,0.517816,1928661.92,2,0.465676,S,S,40.3048,-79.5831


In [21]:
df_sales_by_zip.head(3)

,store_id,zip_5,unique_trans,total_sales,store_trans,store_sales,trans_cum,trans_rank,trans_cum_pctg,sales_cum,sales_rank,sales_cum_pctg,trans_label,sales_label,latitude,longitude,dist_miles
0,1023,15601,6905,1467085.04,17597,4141636.76,6905,1,0.392396,1467085.04,1,0.354228,P,P,40.3048,-79.5831,2.283179
1,1023,15644,2207,461576.88,17597,4141636.76,9112,2,0.517816,1928661.92,2,0.465676,S,S,40.3048,-79.5831,2.576020
2,1023,15642,1501,396914.72,17597,4141636.76,10613,3,0.603114,2325576.64,3,0.561511,S,P,40.3048,-79.5831,7.150137


In [22]:
# only keep zips within X miles (according to business nature)
print("num_miles_to_create_TA: ",num_miles_to_create_TA)

df_output_PS_40_miles=df_sales_by_zip[(df_sales_by_zip['trans_label'].isin(['P','S']))]
df_output_PS_40_miles=df_output_PS_40_miles[df_output_PS_40_miles['dist_miles']<=num_miles_to_create_TA]
df_output_PS_40_miles.shape

num_miles_to_create_TA:  40


(76758, 17)

In [23]:
# Put the zips from the long DF into a value for each row by store for further use

df_store_zip_both=df_output_PS_40_miles[['store_id','trans_label','zip_5','latitude','longitude']].rename(columns={"zip_5":"zip_list_P_and_S"}).drop_duplicates()
df_store_zip_both=df_store_zip_both.groupby(["store_id",'latitude','longitude'])['zip_list_P_and_S'].apply(set).to_frame().reset_index()
df_store_zip_both['zip_list_P_and_S']=df_store_zip_both['zip_list_P_and_S'].apply(lambda x: list(x))

df_store_zip_P=df_output_PS_40_miles[df_output_PS_40_miles['trans_label']=="P"]
df_store_zip_P=df_store_zip_P[['store_id','trans_label','zip_5','latitude','longitude']].rename(columns={"zip_5":"zip_list_P"}).drop_duplicates()
df_store_zip_P=df_store_zip_P.groupby(["store_id",'latitude','longitude'])['zip_list_P'].apply(set).to_frame().reset_index()
df_store_zip_P['zip_list_P']=df_store_zip_P['zip_list_P'].apply(lambda x: list(x))

df_store_zip_S=df_output_PS_40_miles[df_output_PS_40_miles['trans_label']=="S"]
df_store_zip_S=df_store_zip_S[['store_id','trans_label','zip_5','latitude','longitude']].rename(columns={"zip_5":"zip_list_S"}).drop_duplicates()
df_store_zip_S=df_store_zip_S.groupby(["store_id",'latitude','longitude'])['zip_list_S'].apply(set).to_frame().reset_index()
df_store_zip_S['zip_list_S']=df_store_zip_S['zip_list_S'].apply(lambda x: list(x))

df_store_zip=pd.merge(df_store_zip_both,df_store_zip_P,on=["store_id",'latitude','longitude'],how="outer")
df_store_zip=pd.merge(df_store_zip,df_store_zip_S,on=["store_id",'latitude','longitude'],how="outer")


In [24]:
df_store_zip.shape

(567, 6)

In [25]:
df_store_zip.head(5)

,store_id,latitude,longitude,zip_list_P_and_S,zip_list_P,zip_list_S
0,1023,40.304800,-79.583100,"[15627, 15901, 15646, 15462, 15464, 15120, 150...",[15601],"[15627, 15901, 15646, 15462, 15464, 15120, 150..."
1,1027,40.348400,-79.946400,"[15464, 15120, 26047, 15035, 15067, 15314, 150...","[15207, 15227, 15120, 15122]","[15464, 26047, 15035, 15067, 15314, 15062, 152..."
2,1034,40.464103,-79.924700,"[15120, 15739, 15062, 15221, 15126, 15148, 152...","[15201, 15232, 15221, 15208, 15206, 15235]","[15120, 15739, 15062, 15126, 15148, 15230, 151..."
3,1039,41.024466,-80.675370,"[44502, 44272, 15066, 44471, 44657, 44491, 161...","[44512, 44514]","[44502, 44272, 15066, 44471, 44657, 44491, 161..."
4,1047,41.148939,-80.665188,"[44502, 44272, 44471, 44093, 44657, 16112, 444...","[44504, 44505, 44425, 44446, 44515]","[44502, 44272, 44471, 44093, 44657, 16112, 444..."


In [26]:
# Full join to use as iteration (limited size so good to go)

df_store_zip_0=df_store_zip.copy()

df_store_zip.columns=["store_A_"+x for x in df_store_zip.columns.tolist()]
df_store_zip_0.columns=["store_B_"+x for x in df_store_zip_0.columns.tolist()]

df_store_zip['key']=1
df_store_zip_0['key']=1

df_store_zip=pd.merge(df_store_zip,df_store_zip_0,on="key",how="outer")
del df_store_zip_0


df_store_zip=df_store_zip[df_store_zip['store_A_store_id']!=df_store_zip['store_B_store_id']]
df_store_zip=df_store_zip.reset_index()
del df_store_zip['index']
del df_store_zip['key']

gc.collect()

60

In [27]:
# Define to add common zips for 2 list & the distence between stores

def find_intersection(list_1,list_2):
    return list(set(list_1).intersection(set(list_2)))
def dist_miles(lat_1,log_1,lat_2,log_2):
    return haversine((lat_1,log_1),(lat_2,log_2),unit="mi")

In [28]:
df_store_zip.head(2)

,store_A_store_id,store_A_latitude,store_A_longitude,store_A_zip_list_P_and_S,store_A_zip_list_P,store_A_zip_list_S,store_B_store_id,store_B_latitude,store_B_longitude,store_B_zip_list_P_and_S,store_B_zip_list_P,store_B_zip_list_S
0,1023,40.3048,-79.5831,"[15627, 15901, 15646, 15462, 15464, 15120, 150...",[15601],"[15627, 15901, 15646, 15462, 15464, 15120, 150...",1027,40.348400,-79.9464,"[15464, 15120, 26047, 15035, 15067, 15314, 150...","[15207, 15227, 15120, 15122]","[15464, 26047, 15035, 15067, 15314, 15062, 152..."
1,1023,40.3048,-79.5831,"[15627, 15901, 15646, 15462, 15464, 15120, 150...",[15601],"[15627, 15901, 15646, 15462, 15464, 15120, 150...",1034,40.464103,-79.9247,"[15120, 15739, 15062, 15221, 15126, 15148, 152...","[15201, 15232, 15221, 15208, 15206, 15235]","[15120, 15739, 15062, 15126, 15148, 15230, 151..."


In [29]:
# Add overlap percentage based on the common zips compare to the unique zips for 2 stores
# Also add distence

df_store_zip['intersection_both_P_S']=df_store_zip.apply(lambda x: find_intersection(x['store_A_zip_list_P_and_S'],x['store_B_zip_list_P_and_S']),axis=1)

df_store_zip['count_of_overlap_zips']=df_store_zip['intersection_both_P_S'].apply(lambda x: len(x))
df_store_zip['count_of_A_zips']=df_store_zip['store_A_zip_list_P_and_S'].apply(lambda x: len(x))
df_store_zip['count_of_B_zips']=df_store_zip['store_B_zip_list_P_and_S'].apply(lambda x: len(x))
df_store_zip['total_zips']=df_store_zip['count_of_A_zips']+df_store_zip['count_of_B_zips']-df_store_zip['count_of_overlap_zips']

df_store_zip['overlap_pctg']=df_store_zip['count_of_overlap_zips']/df_store_zip['total_zips']

df_store_zip=df_store_zip[df_store_zip['count_of_overlap_zips']>0]

df_store_zip['dist_between_stores']=df_store_zip.apply(lambda x: dist_miles(x['store_A_latitude'],x['store_A_longitude'],x['store_B_latitude'],x['store_B_longitude']),axis=1)
df_store_zip=df_store_zip.reset_index()
del df_store_zip['index']

In [30]:
# Now like this below
df_store_zip.head(2)

,store_A_store_id,store_A_latitude,store_A_longitude,store_A_zip_list_P_and_S,store_A_zip_list_P,store_A_zip_list_S,store_B_store_id,store_B_latitude,store_B_longitude,store_B_zip_list_P_and_S,store_B_zip_list_P,store_B_zip_list_S,intersection_both_P_S,count_of_overlap_zips,count_of_A_zips,count_of_B_zips,total_zips,overlap_pctg,dist_between_stores
0,1023,40.3048,-79.5831,"[15627, 15901, 15646, 15462, 15464, 15120, 150...",[15601],"[15627, 15901, 15646, 15462, 15464, 15120, 150...",1027,40.348400,-79.9464,"[15464, 15120, 26047, 15035, 15067, 15314, 150...","[15207, 15227, 15120, 15122]","[15464, 26047, 15035, 15067, 15314, 15062, 152...","[15464, 15120, 15035, 15067, 15314, 15062, 152...",136,229,188,281,0.483986,19.372330
1,1023,40.3048,-79.5831,"[15627, 15901, 15646, 15462, 15464, 15120, 150...",[15601],"[15627, 15901, 15646, 15462, 15464, 15120, 150...",1034,40.464103,-79.9247,"[15120, 15739, 15062, 15221, 15126, 15148, 152...","[15201, 15232, 15221, 15208, 15206, 15235]","[15120, 15739, 15062, 15126, 15148, 15230, 151...","[15120, 15062, 15221, 15126, 15148, 15213, 152...",107,229,137,259,0.413127,21.079957


In [29]:
# Save it, in case server collapsed... 
df_store_zip.to_csv("df_store_zip_"+str(datetime.datetime.now().date())+".csv",index=False)


In [31]:
# Now each row is a pair of A&B stores, so let's indentify each row

def store_pair_func(store_A,store_B):
    return sorted([store_A,store_B])
df_store_zip_unique_pairs=df_store_zip.copy()
df_store_zip_unique_pairs['store_pair']=df_store_zip_unique_pairs.apply(lambda x: store_pair_func(x['store_A_store_id'],x['store_B_store_id']),axis=1)
df_store_zip_unique_pairs['store_pair_str']=df_store_zip_unique_pairs['store_pair'].astype(str)
df_store_zip_unique_pairs=df_store_zip_unique_pairs.sort_values(['store_pair_str','store_A_store_id','store_B_store_id'])
df_store_zip_unique_pairs=df_store_zip_unique_pairs.drop_duplicates("store_pair_str")
df_store_zip_unique_pairs.to_csv("df_store_zip_unique_pairs_"+str(datetime.datetime.now().date())+".csv",index=False)


In [32]:
# When use the groups of overlap and the distance, we saw the distribution first
# Skip here

df_store_zip_unique_pairs.head(3)

,store_A_store_id,store_A_latitude,store_A_longitude,store_A_zip_list_P_and_S,store_A_zip_list_P,store_A_zip_list_S,store_B_store_id,store_B_latitude,store_B_longitude,store_B_zip_list_P_and_S,store_B_zip_list_P,store_B_zip_list_S,intersection_both_P_S,count_of_overlap_zips,count_of_A_zips,count_of_B_zips,total_zips,overlap_pctg,dist_between_stores,store_pair,store_pair_str
0,1023,40.3048,-79.5831,"[15627, 15901, 15646, 15462, 15464, 15120, 150...",[15601],"[15627, 15901, 15646, 15462, 15464, 15120, 150...",1027,40.348400,-79.94640,"[15464, 15120, 26047, 15035, 15067, 15314, 150...","[15207, 15227, 15120, 15122]","[15464, 26047, 15035, 15067, 15314, 15062, 152...","[15464, 15120, 15035, 15067, 15314, 15062, 152...",136,229,188,281,0.483986,19.372330,"[1023, 1027]","['1023', '1027']"
1,1023,40.3048,-79.5831,"[15627, 15901, 15646, 15462, 15464, 15120, 150...",[15601],"[15627, 15901, 15646, 15462, 15464, 15120, 150...",1034,40.464103,-79.92470,"[15120, 15739, 15062, 15221, 15126, 15148, 152...","[15201, 15232, 15221, 15208, 15206, 15235]","[15120, 15739, 15062, 15126, 15148, 15230, 151...","[15120, 15062, 15221, 15126, 15148, 15213, 152...",107,229,137,259,0.413127,21.079957,"[1023, 1034]","['1023', '1034']"
2,1023,40.3048,-79.5831,"[15627, 15901, 15646, 15462, 15464, 15120, 150...",[15601],"[15627, 15901, 15646, 15462, 15464, 15120, 150...",1039,41.024466,-80.67537,"[44502, 44272, 15066, 44471, 44657, 44491, 161...","[44512, 44514]","[44502, 44272, 15066, 44471, 44657, 44491, 161...",[16066],1,229,124,352,0.002841,75.824736,"[1023, 1039]","['1023', '1039']"


In [33]:
# Harded coded the group scenario

df_store_zip_unique_pairs['overlap_group']=np.where(df_store_zip_unique_pairs['overlap_pctg']>0.2,"20%+",
                                                   np.where(df_store_zip_unique_pairs['overlap_pctg']>0.15,"15%-20%",
                                                           np.where(df_store_zip_unique_pairs['overlap_pctg']>0.1,"10%-15%",
                                                                   np.where(df_store_zip_unique_pairs['overlap_pctg']>0.05,'5%-10%',
                                                                           np.where(df_store_zip_unique_pairs['overlap_pctg']>0.03,'3%-5%',"<3%")
                                                                           )
                                                                   )
                                                           )
                                                   )

df_store_zip_unique_pairs['dist_group']=np.where(df_store_zip_unique_pairs['dist_between_stores']>28,"28+ miles",
                                                   np.where(df_store_zip_unique_pairs['dist_between_stores']>20,"20-28 miles",
                                                           np.where(df_store_zip_unique_pairs['dist_between_stores']>10,"10-20 miles","0-10 miles")
                                                           )
                                                   )

df_summary_overlap_dist_group_pair_count=df_store_zip_unique_pairs.groupby(['overlap_group','dist_group'])['store_pair_str'].count().to_frame().reset_index().rename(columns={"store_pair_str":"pairs_count"})
df_summary_overlap_dist_group_unique_stores=df_store_zip_unique_pairs.groupby(['overlap_group','dist_group'])['store_pair'].sum().to_frame().reset_index().rename(columns={"store_pair":"stores_involved"})

df_store_zip_unique_pairs['zips_P_S_40_both_stores']=df_store_zip_unique_pairs['store_B_zip_list_P_and_S']+df_store_zip_unique_pairs['store_A_zip_list_P_and_S']
df_store_zip_unique_pairs['zips_P_S_40_both_stores']=df_store_zip_unique_pairs['zips_P_S_40_both_stores'].apply(lambda x: list(set(x)))
df_summary_overlap_dist_group_unique_zips=df_store_zip_unique_pairs.groupby(['overlap_group','dist_group'])['zips_P_S_40_both_stores'].sum().to_frame().reset_index().rename(columns={"zips_P_S_40_both_stores":"zips_involved"})

df_summary_overlap_dist_group_unique_zips.head(3)

,overlap_group,dist_group,zips_involved
0,10%-15%,0-10 miles,"[44272, 44319, 44145, 44092, 44087, 44129, 441..."
1,10%-15%,10-20 miles,"[44319, 44643, 44145, 44129, 44606, 44216, 442..."
2,10%-15%,20-28 miles,"[44272, 44319, 44145, 44092, 44087, 44129, 441..."


In [34]:
# Let's summarize and see how they look like
# Just to make sure reasonable

sort_dict={"<3%":1,"3%-5%":2,"5%-10%":3,"10%-15%":4,"15%-20%":5,"20%+":6}

df_summary_pivot_pair=df_summary_overlap_dist_group_pair_count.pivot_table(index="overlap_group",columns='dist_group',values="pairs_count").reset_index()
df_summary_pivot_pair['sort']=df_summary_pivot_pair['overlap_group'].apply(lambda x: sort_dict[x])
df_summary_pivot_pair=df_summary_pivot_pair.sort_values("sort").reset_index()
del df_summary_pivot_pair['sort']
del df_summary_pivot_pair['index']


df_summary_overlap_dist_group_unique_stores['stores_involved']=df_summary_overlap_dist_group_unique_stores['stores_involved'].apply(lambda x: set(x))
df_summary_overlap_dist_group_unique_stores['store_count_involved']=df_summary_overlap_dist_group_unique_stores['stores_involved'].apply(lambda x: len(x))

df_summary_pivot_store=df_summary_overlap_dist_group_unique_stores.pivot_table(index="overlap_group",columns='dist_group',values="store_count_involved").reset_index()
df_summary_pivot_store['sort']=df_summary_pivot_store['overlap_group'].apply(lambda x: sort_dict[x])
df_summary_pivot_store=df_summary_pivot_store.sort_values("sort").reset_index()
del df_summary_pivot_store['sort']
del df_summary_pivot_store['index']




In [35]:
df_summary_pivot_store

dist_group,overlap_group,0-10 miles,10-20 miles,20-28 miles,28+ miles
0,<3%,NaN,NaN,NaN,345.0
1,3%-5%,NaN,NaN,NaN,213.0
2,5%-10%,5.0,2.0,NaN,242.0
3,10%-15%,2.0,2.0,2.0,187.0
4,15%-20%,NaN,NaN,NaN,152.0
5,20%+,421.0,393.0,333.0,342.0


In [36]:
df_summary_overlap_dist_group_unique_zips.head(3)

,overlap_group,dist_group,zips_involved
0,10%-15%,0-10 miles,"[44272, 44319, 44145, 44092, 44087, 44129, 441..."
1,10%-15%,10-20 miles,"[44319, 44643, 44145, 44129, 44606, 44216, 442..."
2,10%-15%,20-28 miles,"[44272, 44319, 44145, 44092, 44087, 44129, 441..."


In [37]:
# summary by zipcode, above is by store
df_summary_overlap_dist_group_unique_zips['zips_involved']=df_summary_overlap_dist_group_unique_zips['zips_involved'].apply(lambda x: set(x))
df_summary_overlap_dist_group_unique_zips['zip_count_involved']=df_summary_overlap_dist_group_unique_zips['zips_involved'].apply(lambda x: len(x))

df_summary_pivot_zip=df_summary_overlap_dist_group_unique_zips.pivot_table(index="overlap_group",columns='dist_group',values="zip_count_involved").reset_index()
df_summary_pivot_zip['sort']=df_summary_pivot_zip['overlap_group'].apply(lambda x: sort_dict[x])
df_summary_pivot_zip=df_summary_pivot_zip.sort_values("sort").reset_index()
del df_summary_pivot_zip['sort']
del df_summary_pivot_zip['index']
df_summary_pivot_zip.head(3)

dist_group,overlap_group,0-10 miles,10-20 miles,20-28 miles,28+ miles
0,<3%,NaN,NaN,NaN,12639.0
1,3%-5%,NaN,NaN,NaN,10586.0
2,5%-10%,235.0,164.0,NaN,10319.0


In [41]:
# QC and review

dist_group=df_store_zip_unique_pairs.groupby(['dist_group'])['store_pair'].sum().to_frame().reset_index().rename(columns={"store_pair":"unique_store_list"})
dist_group['unique_store_list']=dist_group['unique_store_list'].apply(lambda x: list(set(x)))
dist_group['unique_store_count']=dist_group['unique_store_list'].apply(lambda x: len(x))

dist_group_pair_count=df_store_zip_unique_pairs.groupby(['dist_group'])['store_pair'].count().to_frame().reset_index().rename(columns={"store_pair":"pair_counts"})

dist_group=pd.merge(dist_group,dist_group_pair_count,on="dist_group")

In [42]:
dist_group

,dist_group,unique_store_list,unique_store_count,pair_counts
0,0-10 miles,"[2370, 280, 2366, 2771, 1239, 8628, 4158, 4861...",422,556
1,10-20 miles,"[2370, 2347, 280, 2366, 2771, 1239, 2820, 8628...",394,757
2,20-28 miles,"[2370, 2347, 2366, 280, 2169, 1239, 8628, 4158...",334,470
3,28+ miles,"[2370, 280, 2366, 2169, 1239, 2771, 2820, 8628...",516,1985


In [43]:
df_sales_by_zip.head(3)

,store_id,zip_5,unique_trans,total_sales,store_trans,store_sales,trans_cum,trans_rank,trans_cum_pctg,sales_cum,sales_rank,sales_cum_pctg,trans_label,sales_label,latitude,longitude,dist_miles
0,1023,15601,6905,1467085.04,17597,4141636.76,6905,1,0.392396,1467085.04,1,0.354228,P,P,40.3048,-79.5831,2.283179
1,1023,15644,2207,461576.88,17597,4141636.76,9112,2,0.517816,1928661.92,2,0.465676,S,S,40.3048,-79.5831,2.576020
2,1023,15642,1501,396914.72,17597,4141636.76,10613,3,0.603114,2325576.64,3,0.561511,S,P,40.3048,-79.5831,7.150137


In [44]:
# Get the stores and the zips

df_all_P_zips_by_store=df_sales_by_zip[df_sales_by_zip['trans_label']=="P"].groupby(['store_id'])['zip_5'].apply(list).to_frame().reset_index().rename(columns={"zip_5":"all_P_zips"})
df_all_S_zips_by_store=df_sales_by_zip[df_sales_by_zip['trans_label']=="S"].groupby(['store_id'])['zip_5'].apply(list).to_frame().reset_index().rename(columns={"zip_5":"all_S_zips"})

df_40_miles_P_zips_by_store=df_sales_by_zip[(df_sales_by_zip['trans_label']=="P") & (df_sales_by_zip['dist_miles']<=50)].groupby(['store_id'])['zip_5'].apply(list).to_frame().reset_index().rename(columns={"zip_5":"40_miles_P_zips"})
df_40_miles_S_zips_by_store=df_sales_by_zip[(df_sales_by_zip['trans_label']=="S") & (df_sales_by_zip['dist_miles']<=50)].groupby(['store_id'])['zip_5'].apply(list).to_frame().reset_index().rename(columns={"zip_5":"40_miles_S_zips"})

df_P_S_zips_by_store=pd.merge(df_all_P_zips_by_store,df_all_S_zips_by_store,on="store_id",how="outer")
df_P_S_zips_by_store=pd.merge(df_P_S_zips_by_store,df_40_miles_P_zips_by_store,on="store_id",how="outer")
df_P_S_zips_by_store=pd.merge(df_P_S_zips_by_store,df_40_miles_S_zips_by_store,on="store_id",how="outer")
df_P_S_zips_by_store=df_P_S_zips_by_store.fillna("[]")
df_P_S_zips_by_store['all_P_zips']=df_P_S_zips_by_store['all_P_zips'].astype(str).apply(lambda x: eval(x))
df_P_S_zips_by_store['all_S_zips']=df_P_S_zips_by_store['all_S_zips'].astype(str).apply(lambda x: eval(x))
df_P_S_zips_by_store['40_miles_P_zips']=df_P_S_zips_by_store['40_miles_P_zips'].astype(str).apply(lambda x: eval(x))
df_P_S_zips_by_store['40_miles_S_zips']=df_P_S_zips_by_store['40_miles_S_zips'].astype(str).apply(lambda x: eval(x))



In [45]:
print(df_P_S_zips_by_store.shape)
df_P_S_zips_by_store.head(2)

(567, 5)


,store_id,all_P_zips,all_S_zips,40_miles_P_zips,40_miles_S_zips
0,1023,[15601],"[15644, 15642, 15650, 15231, 15401, 17103, 156...",[15601],"[15644, 15642, 15650, 15231, 15401, 15666, 156..."
1,1027,"[15227, 15122, 15120, 15207]","[15236, 15025, 15210, 15102, 15129, 15037, 151...","[15227, 15122, 15120, 15207]","[15236, 15025, 15210, 15102, 15129, 15037, 151..."


In [46]:
df_P_S_zips_by_store['store_id'].nunique()

567

In [52]:
# A few funcs to use later

def dist_of_2_str_stores(pair_stores_str):
    pair_stores_str=eval(pair_stores_str)
    store_a=pair_stores_str[0]
    store_b=pair_stores_str[1]
    dist=haversine([dict_store_lat[store_a],dict_store_lng[store_a]],[dict_store_lat[store_b],dict_store_lng[store_b]],unit="mi")
    return dist

# Only used in below loop
def find_nearest_store(store_id):
    shortest_dist=99999
    nearest_store=np.nan
    for i_store in df_store_address['store_id'].tolist():
        if i_store!=store_id:
            dist=haversine([dict_store_lat[store_id],dict_store_lng[store_id]],[dict_store_lat[i_store],dict_store_lng[i_store]],unit="mi")
            if dist<shortest_dist:
                shortest_dist=dist
                nearest_store=i_store
    return nearest_store, shortest_dist


# The func below is to find out the min, max overlapped stores, given 1 store per row
def find_overlaped_stores(store_id):
    df=df_store_zip_unique_pairs[df_store_zip_unique_pairs['store_pair'].apply(lambda x: store_id in x)]
    df=df.sort_values('overlap_pctg',ascending=True).reset_index()
    min_overlap_pctg=df.loc[0,'overlap_pctg']
    min_overlap_store_pair_str=df.loc[0,'store_pair_str']
    
    max_overlap_pctg=df.loc[len(df)-1,'overlap_pctg']
    max_overlap_store_pair_str=df.loc[len(df)-1,'store_pair_str']
    
    return min_overlap_pctg,min_overlap_store_pair_str,max_overlap_pctg,max_overlap_store_pair_str

In [47]:
# clean zips and extrac the store address to put into the summary
df_store_included['zip_cd']=df_store_included['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_address=df_store_included[['store_num','address_1','city','state','zip_cd','latitude','longitude']]
df_store_address=df_store_address.rename(columns={"store_num":"store_id"})

In [50]:
dist_group

,dist_group,unique_store_list,unique_store_count,pair_counts
0,0-10 miles,"[2370, 280, 2366, 2771, 1239, 8628, 4158, 4861...",422,556
1,10-20 miles,"[2370, 2347, 280, 2366, 2771, 1239, 2820, 8628...",394,757
2,20-28 miles,"[2370, 2347, 2366, 280, 2169, 1239, 8628, 4158...",334,470
3,28+ miles,"[2370, 280, 2366, 2169, 1239, 2771, 2820, 8628...",516,1985


In [53]:
# Writer out the stores by each distance group and the overlap group
# To review the excel

nearest_store_dist_criteria_dict={"0-10 miles":0,"10-20 miles":10,"20-28 miles":20,"28+ miles":28}

writer=pd.ExcelWriter(output_folder+"GY_new_TA_pair_dist_table_summary_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
dist_group.to_excel(writer,"pairs_and_stores_by_pair_dist",index=False)

ordered_cols=['dist_group','store_id','address_1','city','state','zip_cd','latitude','longitude',
              'all_P_zips','all_S_zips','All_P&S_zips','40_miles_P_zips','40_miles_S_zips','All_P&S_zips_40_miles',
              'nearest_store','dist_to_nearest_store',
              'max_overlap_store','max_overlap_pctg','dist_max_overlap_pairs',
              'min_overlap_pctg','min_overlap_store','dist_min_overlap_pairs']

all_paired_group_store_df=pd.DataFrame(columns=ordered_cols)
all_single_store_df=pd.DataFrame(columns=['nearest_store_dist_criteria']+ordered_cols)
all_high_overlap_store_df=pd.DataFrame(columns=ordered_cols)

for dist,group in dist_group.groupby("dist_group"):
    store_list_in_dist_group=group['unique_store_list'].values[0]
    df_store_zip_unique_pairs_removed=df_store_zip_unique_pairs[(~df_store_zip_unique_pairs['store_A_store_id'].isin(store_list_in_dist_group)) &\
                                                                 (~df_store_zip_unique_pairs['store_B_store_id'].isin(store_list_in_dist_group))]

    df_removed_pairs_rows=df_store_zip_unique_pairs[df_store_zip_unique_pairs['dist_group']==dist]
    df_store_zip_unique_pairs_removed_stores=df_store_zip_unique_pairs[(df_store_zip_unique_pairs['store_A_store_id'].isin(store_list_in_dist_group)) |\
                                                                 (df_store_zip_unique_pairs['store_B_store_id'].isin(store_list_in_dist_group))]
    
    df_removed_stores_1=pd.DataFrame({"store_id":store_list_in_dist_group})
    df_removed_stores_1=pd.merge(df_removed_stores_1,df_store_address,on="store_id",how="left")
    df_removed_stores_1=pd.merge(df_removed_stores_1,df_P_S_zips_by_store,on="store_id",how="left")
    
    df_removed_stores_1['nearest_store']=df_removed_stores_1['store_id'].apply(lambda x: find_nearest_store(x)[0])
    df_removed_stores_1['dist_to_nearest_store']=df_removed_stores_1['store_id'].apply(lambda x: find_nearest_store(x)[1])

    df_removed_stores_1['min_overlap_pctg']=df_removed_stores_1['store_id'].apply(lambda x: find_overlaped_stores(x)[0])
    df_removed_stores_1['min_overlap_store']=df_removed_stores_1['store_id'].apply(lambda x: find_overlaped_stores(x)[1])
    df_removed_stores_1['max_overlap_pctg']=df_removed_stores_1['store_id'].apply(lambda x: find_overlaped_stores(x)[2])
    df_removed_stores_1['max_overlap_store']=df_removed_stores_1['store_id'].apply(lambda x: find_overlaped_stores(x)[3])

    df_removed_stores_1['All_P&S_zips']=df_removed_stores_1['all_P_zips']+df_removed_stores_1['all_S_zips']
    df_removed_stores_1['All_P&S_zips_40_miles']=df_removed_stores_1['40_miles_P_zips']+df_removed_stores_1['40_miles_S_zips']

    df_removed_stores_1['dist_min_overlap_pairs']=df_removed_stores_1['min_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))
    df_removed_stores_1['dist_max_overlap_pairs']=df_removed_stores_1['max_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))
    df_removed_stores_1['dist_group']=dist
        
    df_removed_stores_1=df_removed_stores_1[ordered_cols]
    df_removed_stores_1=df_removed_stores_1.sort_values("dist_to_nearest_store",ascending=False)
    
    
    nearest_store_dist_criteria=nearest_store_dist_criteria_dict[dist]
    single_store_df=df_removed_stores_1[(df_removed_stores_1['dist_to_nearest_store']>nearest_store_dist_criteria) &\
                                        (df_removed_stores_1['max_overlap_pctg']<=0.15)]
    single_store_df['nearest_store_dist_criteria']=nearest_store_dist_criteria
    
    # Cretio for high overlap below
    high_overlap_store_df=df_removed_stores_1[(df_removed_stores_1['dist_to_nearest_store']<=10) &\
                                        (df_removed_stores_1['max_overlap_pctg']>=0.5)]
       
    df_removed_stores_1.to_excel(writer,dist+"_all_stores",index=False)
    
    
    all_paired_group_store_df=all_paired_group_store_df.append(df_removed_stores_1)
    all_single_store_df=all_single_store_df.append(single_store_df)

    all_high_overlap_store_df=all_high_overlap_store_df.append(high_overlap_store_df)
    

def clean_list_col(df):
    for col in df.columns.tolist():
        if df[col].apply(lambda x: type(x)).unique()[0]==list:
            df[col]=df[col].astype(str)
    return df
            
all_single_store_df=clean_list_col(all_single_store_df)
all_paired_group_store_df=clean_list_col(all_paired_group_store_df)
all_high_overlap_store_df=clean_list_col(all_high_overlap_store_df)

all_single_store_df=all_single_store_df.drop_duplicates()
all_paired_group_store_df=all_paired_group_store_df.drop_duplicates()
all_high_overlap_store_df=all_high_overlap_store_df.drop_duplicates()

all_single_store_df=all_single_store_df[['nearest_store_dist_criteria']+ordered_cols]
all_single_store_df.to_excel(writer,"all_single_store_df",index=False)
all_paired_group_store_df.to_excel(writer,"all_paired_group_store_df",index=False)
all_high_overlap_store_df.to_excel(writer,"all_high_overlap_store_df",index=False)

all_unique_high_overlap_stores=all_high_overlap_store_df.copy()
del all_high_overlap_store_df['dist_group']
all_high_overlap_store_df=all_high_overlap_store_df.drop_duplicates()
all_high_overlap_store_df=all_high_overlap_store_df.sort_values(["state",'city','dist_to_nearest_store'])
all_high_overlap_store_df.to_excel(writer,"unique_all_high_overlap_store",index=False)

df_summary_high_overlap_store_st=all_high_overlap_store_df.groupby(all_high_overlap_store_df['state'])['store_id'].nunique().to_frame().reset_index()
df_summary_high_overlap_store_st.to_excel(writer,"summary_high_overlap_st_count",index=False)
writer.save()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [54]:
all_paired_group_store_df.head(4)

,dist_group,store_id,address_1,city,state,zip_cd,latitude,longitude,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,max_overlap_store,max_overlap_pctg,dist_max_overlap_pairs,min_overlap_pctg,min_overlap_store,dist_min_overlap_pairs
272,0-10 miles,8026,121 ALAMAHA STREET,KAHULUI,HI,96732,20.889472,-156.462388,"['96793', '96768']","['96732', '96708', '96790', '96753', '96761', ...","['96793', '96768', '96732', '96708', '96790', ...","['96793', '96768']","['96732', '96708', '96790', '96753', '96761', ...","['96793', '96768', '96732', '96708', '96790', ...",8027,9.997319,"['8026', '8027']",0.933333,9.997319,0.933333,"['8026', '8027']",9.997319
90,0-10 miles,8027,105 HALEKUAI ST,KIHEI,HI,96753,20.745132,-156.451585,['96753'],"['96793', '96768', '93436', '96732', '96790', ...","['96753', '96793', '96768', '93436', '96732', ...",['96753'],"['96793', '96768', '96732', '96790', '96761', ...","['96753', '96793', '96768', '96732', '96790', ...",8026,9.997319,"['8026', '8027']",0.933333,9.997319,0.933333,"['8026', '8027']",9.997319
359,0-10 miles,2461,131 GATEWAY CIRCLE,JACKSONVILLE,FL,32259,30.063017,-81.506737,['32092'],"['32259', '32081', '32095', '32258', '32256', ...","['32092', '32259', '32081', '32095', '32258', ...",['32092'],"['32259', '32081', '32095', '32258', '32256', ...","['32092', '32259', '32081', '32095', '32258', ...",2448,9.968136,"['2448', '2461']",0.762500,9.968136,0.029851,"['2461', '2586']",61.646134
387,0-10 miles,8163,30405 BRYANT DRIVE,EVERGREEN,CO,80439,39.650699,-105.344868,['80439'],"['80437', '80433', '80452', '80401', '80421', ...","['80439', '80437', '80433', '80452', '80401', ...",['80439'],"['80437', '80433', '80452', '80401', '80421', ...","['80439', '80437', '80433', '80452', '80401', ...",8120,9.965407,"['8120', '8163']",0.750000,9.965407,0.010363,"['8163', '8167']",63.549136


In [55]:
# Format the df and add columns

def remove_the_hold_store(pair_list,hold_store):
    y=pair_list.copy()
    y.remove(hold_store)
    return y[0]

all_paired_group_store_df['min_overlap_store']=all_paired_group_store_df['min_overlap_store'].apply(lambda x: eval(x))
all_paired_group_store_df['max_overlap_store']=all_paired_group_store_df['max_overlap_store'].apply(lambda x: eval(x))

all_paired_group_store_df['all_P_zips']=all_paired_group_store_df['all_P_zips'].apply(lambda x: eval(x))
all_paired_group_store_df['all_S_zips']=all_paired_group_store_df['all_S_zips'].apply(lambda x: eval(x))
all_paired_group_store_df['All_P&S_zips']=all_paired_group_store_df['All_P&S_zips'].apply(lambda x: eval(x))
all_paired_group_store_df['40_miles_P_zips']=all_paired_group_store_df['40_miles_P_zips'].apply(lambda x: eval(x))
all_paired_group_store_df['40_miles_S_zips']=all_paired_group_store_df['40_miles_S_zips'].apply(lambda x: eval(x))
all_paired_group_store_df['All_P&S_zips_40_miles']=all_paired_group_store_df['All_P&S_zips_40_miles'].apply(lambda x: eval(x))

all_paired_group_store_df['min_paired_store']=all_paired_group_store_df.apply(lambda x: remove_the_hold_store(x['min_overlap_store'],x['store_id']),axis=1)
all_paired_group_store_df['max_paired_store']=all_paired_group_store_df.apply(lambda x: remove_the_hold_store(x['max_overlap_store'],x['store_id']),axis=1)

all_paired_group_store_df.head(3)

,dist_group,store_id,address_1,city,state,zip_cd,latitude,longitude,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,max_overlap_store,max_overlap_pctg,dist_max_overlap_pairs,min_overlap_pctg,min_overlap_store,dist_min_overlap_pairs,min_paired_store,max_paired_store
272,0-10 miles,8026,121 ALAMAHA STREET,KAHULUI,HI,96732,20.889472,-156.462388,"[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...","[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...",8027,9.997319,"[8026, 8027]",0.933333,9.997319,0.933333,"[8026, 8027]",9.997319,8027,8027
90,0-10 miles,8027,105 HALEKUAI ST,KIHEI,HI,96753,20.745132,-156.451585,[96753],"[96793, 96768, 93436, 96732, 96790, 96761, 967...","[96753, 96793, 96768, 93436, 96732, 96790, 967...",[96753],"[96793, 96768, 96732, 96790, 96761, 96733, 967...","[96753, 96793, 96768, 96732, 96790, 96761, 967...",8026,9.997319,"[8026, 8027]",0.933333,9.997319,0.933333,"[8026, 8027]",9.997319,8026,8026
359,0-10 miles,2461,131 GATEWAY CIRCLE,JACKSONVILLE,FL,32259,30.063017,-81.506737,[32092],"[32259, 32081, 32095, 32258, 32256, 32084, 322...","[32092, 32259, 32081, 32095, 32258, 32256, 320...",[32092],"[32259, 32081, 32095, 32258, 32256, 32084, 322...","[32092, 32259, 32081, 32095, 32258, 32256, 320...",2448,9.968136,"[2448, 2461]",0.762500,9.968136,0.029851,"[2461, 2586]",61.646134,2586,2448


# Collapse

In [56]:
df_P_S_zips_by_store.head(2)

,store_id,all_P_zips,all_S_zips,40_miles_P_zips,40_miles_S_zips
0,1023,[15601],"[15644, 15642, 15650, 15231, 15401, 17103, 156...",[15601],"[15644, 15642, 15650, 15231, 15401, 15666, 156..."
1,1027,"[15227, 15122, 15120, 15207]","[15236, 15025, 15210, 15102, 15129, 15037, 151...","[15227, 15122, 15120, 15207]","[15236, 15025, 15210, 15102, 15129, 15037, 151..."


In [57]:
all_paired_group_store_df.head(2)

,dist_group,store_id,address_1,city,state,zip_cd,latitude,longitude,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,max_overlap_store,max_overlap_pctg,dist_max_overlap_pairs,min_overlap_pctg,min_overlap_store,dist_min_overlap_pairs,min_paired_store,max_paired_store
272,0-10 miles,8026,121 ALAMAHA STREET,KAHULUI,HI,96732,20.889472,-156.462388,"[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...","[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...",8027,9.997319,"[8026, 8027]",0.933333,9.997319,0.933333,"[8026, 8027]",9.997319,8027,8027
90,0-10 miles,8027,105 HALEKUAI ST,KIHEI,HI,96753,20.745132,-156.451585,[96753],"[96793, 96768, 93436, 96732, 96790, 96761, 967...","[96753, 96793, 96768, 93436, 96732, 96790, 967...",[96753],"[96793, 96768, 96732, 96790, 96761, 96733, 967...","[96753, 96793, 96768, 96732, 96790, 96761, 967...",8026,9.997319,"[8026, 8027]",0.933333,9.997319,0.933333,"[8026, 8027]",9.997319,8026,8026


In [58]:
# Merge in the zip list for the max, min, nearest stores, for each holding store per row
store_PS_40=df_P_S_zips_by_store[['store_id','40_miles_P_zips','40_miles_S_zips']]
store_PS_40['all_PS_40']=store_PS_40['40_miles_P_zips']+store_PS_40['40_miles_S_zips']
store_PS_40['all_PS_40']=store_PS_40['all_PS_40'].apply(lambda x: list(set(x)))
store_PS_40=store_PS_40[['store_id','all_PS_40']]

store_PS_40_min=store_PS_40.rename(columns={"store_id":"min_paired_store","all_PS_40":"min_store_all_PS_40"})
store_PS_40_max=store_PS_40.rename(columns={"store_id":"max_paired_store","all_PS_40":"max_store_all_PS_40"})
store_PS_40_near=store_PS_40.rename(columns={"store_id":"nearest_store","all_PS_40":"near_store_all_PS_40"})

all_paired_group_store_df=pd.merge(all_paired_group_store_df,store_PS_40_min,on="min_paired_store")
all_paired_group_store_df=pd.merge(all_paired_group_store_df,store_PS_40_max,on="max_paired_store")
all_paired_group_store_df=pd.merge(all_paired_group_store_df,store_PS_40_near,on="nearest_store")



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [59]:
# func to see the overlap percentage for each of a pair
def intersection_2_list(list_1,list_2):
    inter_list=list(set(list_1).intersection(set(list_2)))
    inter_pctg_1=len(inter_list)/len(list_1)
    inter_pctg_2=len(inter_list)/len(list_2)
    return inter_list,inter_pctg_1,inter_pctg_2


In [60]:
all_paired_group_store_df.head(2)

,dist_group,store_id,address_1,city,state,zip_cd,latitude,longitude,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,max_overlap_store,max_overlap_pctg,dist_max_overlap_pairs,min_overlap_pctg,min_overlap_store,dist_min_overlap_pairs,min_paired_store,max_paired_store,min_store_all_PS_40,max_store_all_PS_40,near_store_all_PS_40
0,0-10 miles,8026,121 ALAMAHA STREET,KAHULUI,HI,96732,20.889472,-156.462388,"[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...","[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...",8027,9.997319,"[8026, 8027]",0.933333,9.997319,0.933333,"[8026, 8027]",9.997319,8027,8027,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967..."
1,0-10 miles,8027,105 HALEKUAI ST,KIHEI,HI,96753,20.745132,-156.451585,[96753],"[96793, 96768, 93436, 96732, 96790, 96761, 967...","[96753, 96793, 96768, 93436, 96732, 96790, 967...",[96753],"[96793, 96768, 96732, 96790, 96761, 96733, 967...","[96753, 96793, 96768, 96732, 96790, 96761, 967...",8026,9.997319,"[8026, 8027]",0.933333,9.997319,0.933333,"[8026, 8027]",9.997319,8026,8026,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967..."


In [61]:
# Apply func above
all_paired_group_store_df['nearest_intersection_zips']=all_paired_group_store_df.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['near_store_all_PS_40'])[0],axis=1)
all_paired_group_store_df['nearest_pair_overlap_with_hold']=all_paired_group_store_df.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['near_store_all_PS_40'])[1],axis=1)
all_paired_group_store_df['nearest_pair_overlap_with_nearest']=all_paired_group_store_df.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['near_store_all_PS_40'])[2],axis=1)

all_paired_group_store_df['max_intersection_zips']=all_paired_group_store_df.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['max_store_all_PS_40'])[0],axis=1)
all_paired_group_store_df['max_pair_overlap_with_hold']=all_paired_group_store_df.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['max_store_all_PS_40'])[1],axis=1)
all_paired_group_store_df['max_pair_overlap_with_max']=all_paired_group_store_df.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['max_store_all_PS_40'])[2],axis=1)

all_paired_group_store_df['min_intersection_zips']=all_paired_group_store_df.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['min_store_all_PS_40'])[0],axis=1)
all_paired_group_store_df['min_pair_overlap_with_hold']=all_paired_group_store_df.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['min_store_all_PS_40'])[1],axis=1)
all_paired_group_store_df['min_pair_overlap_with_min']=all_paired_group_store_df.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['min_store_all_PS_40'])[2],axis=1)


In [62]:
# Check no empty store
all_paired_group_store_df[all_paired_group_store_df['near_store_all_PS_40'].apply(len)==0]

,dist_group,store_id,address_1,city,state,zip_cd,latitude,longitude,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,max_overlap_store,max_overlap_pctg,dist_max_overlap_pairs,min_overlap_pctg,min_overlap_store,dist_min_overlap_pairs,min_paired_store,max_paired_store,min_store_all_PS_40,max_store_all_PS_40,near_store_all_PS_40,nearest_intersection_zips,nearest_pair_overlap_with_hold,nearest_pair_overlap_with_nearest,max_intersection_zips,max_pair_overlap_with_hold,max_pair_overlap_with_max,min_intersection_zips,min_pair_overlap_with_hold,min_pair_overlap_with_min


In [63]:
# rerange
col_list=['dist_group','store_id','address_1','city','state','zip_cd','latitude','longitude',
          'all_P_zips','all_S_zips','All_P&S_zips','40_miles_P_zips','40_miles_S_zips','All_P&S_zips_40_miles',
          'nearest_store','dist_to_nearest_store','near_store_all_PS_40','nearest_intersection_zips','nearest_pair_overlap_with_hold','nearest_pair_overlap_with_nearest',
          'max_overlap_store','max_overlap_pctg','dist_max_overlap_pairs','max_paired_store','max_store_all_PS_40',
          'max_intersection_zips','max_pair_overlap_with_hold','max_pair_overlap_with_max',
          'min_overlap_store','min_overlap_pctg','dist_min_overlap_pairs','min_paired_store','min_store_all_PS_40',
          'min_intersection_zips','min_pair_overlap_with_hold','min_pair_overlap_with_min']
all_paired_group_store_df=all_paired_group_store_df[col_list]
all_paired_group_store_df.shape

(1666, 36)

In [64]:
# Combine P & S

df_removed_stores_1['All_P&S_zips']=df_removed_stores_1['all_P_zips']+df_removed_stores_1['all_S_zips']
df_removed_stores_1['All_P&S_zips_40_miles']=df_removed_stores_1['40_miles_P_zips']+df_removed_stores_1['40_miles_S_zips']


In [65]:
all_paired_group_store_df.head(3)

,dist_group,store_id,address_1,city,state,zip_cd,latitude,longitude,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,near_store_all_PS_40,nearest_intersection_zips,nearest_pair_overlap_with_hold,nearest_pair_overlap_with_nearest,max_overlap_store,max_overlap_pctg,dist_max_overlap_pairs,max_paired_store,max_store_all_PS_40,max_intersection_zips,max_pair_overlap_with_hold,max_pair_overlap_with_max,min_overlap_store,min_overlap_pctg,dist_min_overlap_pairs,min_paired_store,min_store_all_PS_40,min_intersection_zips,min_pair_overlap_with_hold,min_pair_overlap_with_min
0,0-10 miles,8026,121 ALAMAHA STREET,KAHULUI,HI,96732,20.889472,-156.462388,"[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...","[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...",8027,9.997319,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",0.823529,1.000000,"[8026, 8027]",0.933333,9.997319,8027,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",0.823529,1.000000,"[8026, 8027]",0.933333,9.997319,8027,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",0.823529,1.000000
1,0-10 miles,8027,105 HALEKUAI ST,KIHEI,HI,96753,20.745132,-156.451585,[96753],"[96793, 96768, 93436, 96732, 96790, 96761, 967...","[96753, 96793, 96768, 93436, 96732, 96790, 967...",[96753],"[96793, 96768, 96732, 96790, 96761, 96733, 967...","[96753, 96793, 96768, 96732, 96790, 96761, 967...",8026,9.997319,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",1.000000,0.823529,"[8026, 8027]",0.933333,9.997319,8026,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",1.000000,0.823529,"[8026, 8027]",0.933333,9.997319,8026,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",1.000000,0.823529
2,0-10 miles,2461,131 GATEWAY CIRCLE,JACKSONVILLE,FL,32259,30.063017,-81.506737,[32092],"[32259, 32081, 32095, 32258, 32256, 32084, 322...","[32092, 32259, 32081, 32095, 32258, 32256, 320...",[32092],"[32259, 32081, 32095, 32258, 32256, 32084, 322...","[32092, 32259, 32081, 32095, 32258, 32256, 320...",2448,9.968136,"[32086, 32040, 32241, 32244, 32234, 32046, 322...","[32068, 32259, 32086, 32033, 32202, 32189, 320...",0.819277,0.883117,"[2448, 2461]",0.762500,9.968136,2448,"[32086, 32040, 32241, 32244, 32234, 32046, 322...","[32068, 32259, 32086, 32033, 32202, 32189, 320...",0.819277,0.883117,"[2461, 2586]",0.029851,61.646134,2586,"[34478, 32697, 32113, 32025, 32134, 32234, 344...","[32068, 32656, 32043, 32063, 32177, 32640, 321...",0.132530,0.125000


In [66]:
# Sets of the conditions below to determine the initial core of TAs

In [67]:
# A

# list_all_stores_in_single_TA_1 -- no overlap at all
list_all_stores_in_single_TA_1=df_P_S_zips_by_store[~df_P_S_zips_by_store['store_id'].isin(all_paired_group_store_df['store_id'].tolist())]
list_all_stores_in_single_TA_1=list_all_stores_in_single_TA_1['store_id'].unique().tolist()
print("list_all_stores_in_single_TA_1",len(list_all_stores_in_single_TA_1))
df_no_overlap_at_all=pd.DataFrame({"store_id":list_all_stores_in_single_TA_1})
df_no_overlap_at_all=pd.merge(df_no_overlap_at_all,df_store_address,on="store_id",how="left")
df_no_overlap_at_all=pd.merge(df_no_overlap_at_all,df_P_S_zips_by_store,on="store_id",how="left")
df_no_overlap_at_all['All_P&S_zips']=df_no_overlap_at_all['all_P_zips']+df_no_overlap_at_all['all_S_zips']
df_no_overlap_at_all['All_P&S_zips_40_miles']=df_no_overlap_at_all['40_miles_P_zips']+df_no_overlap_at_all['40_miles_S_zips']

# list_all_stores_in_single_TA_2 -- low oeverlap
list_all_stores_in_single_TA_2=all_paired_group_store_df[(all_paired_group_store_df['nearest_store']==all_paired_group_store_df['max_paired_store']) &\
                                                         (all_paired_group_store_df['nearest_store']==all_paired_group_store_df['min_paired_store']) &\
                                                         (all_paired_group_store_df['max_overlap_pctg']<0.15) &\
                                                         (all_paired_group_store_df['nearest_pair_overlap_with_hold']<0.5) &\
                                                         (all_paired_group_store_df['nearest_pair_overlap_with_nearest']<0.5) &\
                                                         (all_paired_group_store_df['max_pair_overlap_with_hold']<0.5) &\
                                                         (all_paired_group_store_df['max_pair_overlap_with_max']<0.5) &\
                                                         (all_paired_group_store_df['min_pair_overlap_with_hold']<0.5) &\
                                                         (all_paired_group_store_df['min_pair_overlap_with_min']<0.5)]['store_id'].unique().tolist()
print("list_all_stores_in_single_TA_2",len(list_all_stores_in_single_TA_2))
col_single_store_TA=all_paired_group_store_df.columns.tolist()
df_all_single_store_TA=all_paired_group_store_df[all_paired_group_store_df['store_id'].isin(list_all_stores_in_single_TA_2)]
df_all_single_store_TA=df_all_single_store_TA.append(df_no_overlap_at_all)[col_single_store_TA]

already_defined_stores=df_all_single_store_TA['store_id'].unique().tolist()
print("already_defined_stores,",len(already_defined_stores))

list_all_stores_in_single_TA_1 16
list_all_stores_in_single_TA_2 5
already_defined_stores, 21


In [68]:
# To see if any new stores that without store or in store list (not applied here for this client GY)
all_included_P_S_stores=df_P_S_zips_by_store['store_id'].unique().tolist()

new_stores=df_store_address[~df_store_address['store_id'].isin(all_included_P_S_stores)]

new_stores.shape

(0, 7)

In [69]:
new_stores

,store_id,address_1,city,state,zip_cd,latitude,longitude


In [70]:
# For next step to remove already defined stores
def identify_stores_already_asigned(stores_per_row,all_defined_list):
    # y_in_defined_list=[x for x in stores_per_row if x in all_defined_list]
    y_notin_defined_list=[x for x in stores_per_row if x not in all_defined_list]
    
    return y_notin_defined_list

In [71]:
# Format str into list

all_paired_group_store_df['original_combined_stores_4']="["+"'"+all_paired_group_store_df['store_id']+"'"+", "+"'"+all_paired_group_store_df['nearest_store']+"'"+", "+"'"+all_paired_group_store_df['max_paired_store']+"'"+", "+"'"+all_paired_group_store_df['min_paired_store']+"'"+"]"
all_paired_group_store_df['original_combined_stores_4']=all_paired_group_store_df['original_combined_stores_4'].apply(lambda x: list(set(eval(x))))

all_paired_group_store_df['original_combined_stores_3']="["+"'"+all_paired_group_store_df['store_id']+"'"+", "+"'"+all_paired_group_store_df['nearest_store']+"'"+", "+"'"+all_paired_group_store_df['max_paired_store']+"'"+"]"
all_paired_group_store_df['original_combined_stores_3']=all_paired_group_store_df['original_combined_stores_3'].apply(lambda x: list(set(eval(x))))

all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['original_combined_stores_4']
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['original_combined_stores_3']

all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['cleaned_combined_stores_4'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['cleaned_combined_stores_3'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


In [72]:
all_paired_group_store_df.head(3)

,dist_group,store_id,address_1,city,state,zip_cd,latitude,longitude,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,near_store_all_PS_40,nearest_intersection_zips,nearest_pair_overlap_with_hold,nearest_pair_overlap_with_nearest,max_overlap_store,max_overlap_pctg,dist_max_overlap_pairs,max_paired_store,max_store_all_PS_40,max_intersection_zips,max_pair_overlap_with_hold,max_pair_overlap_with_max,min_overlap_store,min_overlap_pctg,dist_min_overlap_pairs,min_paired_store,min_store_all_PS_40,min_intersection_zips,min_pair_overlap_with_hold,min_pair_overlap_with_min,original_combined_stores_4,original_combined_stores_3,cleaned_combined_stores_4,cleaned_combined_stores_3
0,0-10 miles,8026,121 ALAMAHA STREET,KAHULUI,HI,96732,20.889472,-156.462388,"[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...","[96793, 96768]","[96732, 96708, 96790, 96753, 96761, 96779, 967...","[96793, 96768, 96732, 96708, 96790, 96753, 967...",8027,9.997319,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",0.823529,1.000000,"[8026, 8027]",0.933333,9.997319,8027,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",0.823529,1.000000,"[8026, 8027]",0.933333,9.997319,8027,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",0.823529,1.000000,"[8027, 8026]","[8027, 8026]","[8027, 8026]","[8027, 8026]"
1,0-10 miles,8027,105 HALEKUAI ST,KIHEI,HI,96753,20.745132,-156.451585,[96753],"[96793, 96768, 93436, 96732, 96790, 96761, 967...","[96753, 96793, 96768, 93436, 96732, 96790, 967...",[96753],"[96793, 96768, 96732, 96790, 96761, 96733, 967...","[96753, 96793, 96768, 96732, 96790, 96761, 967...",8026,9.997319,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",1.000000,0.823529,"[8026, 8027]",0.933333,9.997319,8026,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",1.000000,0.823529,"[8026, 8027]",0.933333,9.997319,8026,"[96732, 96761, 96733, 96779, 96788, 96768, 967...","[96732, 96761, 96733, 96779, 96788, 96768, 967...",1.000000,0.823529,"[8027, 8026]","[8027, 8026]","[8027, 8026]","[8027, 8026]"
2,0-10 miles,2461,131 GATEWAY CIRCLE,JACKSONVILLE,FL,32259,30.063017,-81.506737,[32092],"[32259, 32081, 32095, 32258, 32256, 32084, 322...","[32092, 32259, 32081, 32095, 32258, 32256, 320...",[32092],"[32259, 32081, 32095, 32258, 32256, 32084, 322...","[32092, 32259, 32081, 32095, 32258, 32256, 320...",2448,9.968136,"[32086, 32040, 32241, 32244, 32234, 32046, 322...","[32068, 32259, 32086, 32033, 32202, 32189, 320...",0.819277,0.883117,"[2448, 2461]",0.762500,9.968136,2448,"[32086, 32040, 32241, 32244, 32234, 32046, 322...","[32068, 32259, 32086, 32033, 32202, 32189, 320...",0.819277,0.883117,"[2461, 2586]",0.029851,61.646134,2586,"[34478, 32697, 32113, 32025, 32134, 32234, 344...","[32068, 32656, 32043, 32063, 32177, 32640, 321...",0.132530,0.125000,"[2586, 2448, 2461]","[2448, 2461]","[2586, 2448, 2461]","[2448, 2461]"


# T1- >=50% overlap

In [73]:
# B - T1 >=50%
# merge 1: 4 stores - hold, max&min, nearest 10 miles


df_B_all_merge_4_40_above=all_paired_group_store_df[(all_paired_group_store_df['dist_to_nearest_store']<=10) &\
                                         (all_paired_group_store_df['max_overlap_pctg']>=0.5) &\
                                         (all_paired_group_store_df['min_overlap_pctg']>=0.5)]
df_B_all_merge_4_40_above['result_combined_stores']=df_B_all_merge_4_40_above['cleaned_combined_stores_4']

list_df_B_all_merge_4_40_above_involved_stores=list(set(df_B_all_merge_4_40_above['result_combined_stores'].sum()))
print('list_df_B_all_merge_4_40_above_involved_stores',len(list_df_B_all_merge_4_40_above_involved_stores))
#### 
already_defined_stores=list(set(already_defined_stores+list_df_B_all_merge_4_40_above_involved_stores))
print("already_defined_stores",len(already_defined_stores))

######
# Clean
all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['cleaned_combined_stores_4'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['cleaned_combined_stores_3'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_B_all_merge_4_40_above_involved_stores 38
already_defined_stores 59


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [74]:
# C - T1 >=50%
# merge 2: 3 stores - hold, max|min, nearest 10 miles

df_C_all_merge_3_40_above=all_paired_group_store_df[(all_paired_group_store_df['dist_to_nearest_store']<=10) &\
                                           ((all_paired_group_store_df['max_overlap_pctg']>=0.5) | (all_paired_group_store_df['min_overlap_pctg']>=0.5))]
df_C_all_merge_3_40_above['result_combined_stores']=df_C_all_merge_3_40_above['cleaned_combined_stores_3']

list_df_C_all_merge_3_40_above_involved_stores=list(set(df_C_all_merge_3_40_above['result_combined_stores'].sum()))
print('list_df_C_all_merge_3_40_above_involved_stores',len(list_df_C_all_merge_3_40_above_involved_stores))

####
already_defined_stores=list(set(already_defined_stores+list_df_C_all_merge_3_40_above_involved_stores))
print("already_defined_stores",len(already_defined_stores))

######
# Clean
all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['cleaned_combined_stores_4'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['cleaned_combined_stores_3'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_C_all_merge_3_40_above_involved_stores 393
already_defined_stores 452


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [75]:
# D - T1 >=50%
# merge 3: 4 stores - hold, max&min, nearest 10-28 miles


df_D_all_merge_4_40_above=all_paired_group_store_df[(all_paired_group_store_df['dist_to_nearest_store']>10) &\
                                           (all_paired_group_store_df['dist_to_nearest_store']<=28) &\
                                           (all_paired_group_store_df['max_overlap_pctg']>=0.5) &\
                                           (all_paired_group_store_df['min_overlap_pctg']>=0.5)]
df_D_all_merge_4_40_above['result_combined_stores']=df_D_all_merge_4_40_above['cleaned_combined_stores_4']
if len(df_D_all_merge_4_40_above)>0:
    list_df_D_all_merge_4_40_above_involved_stores=list(set(df_D_all_merge_4_40_above['result_combined_stores'].sum()))
else:
    list_df_D_all_merge_4_40_above_involved_stores=[]
print('list_df_D_all_merge_4_40_above_involved_stores',len(list_df_D_all_merge_4_40_above_involved_stores))
#### 
already_defined_stores=list(set(already_defined_stores+list_df_D_all_merge_4_40_above_involved_stores))
print("already_defined_stores",len(already_defined_stores))

######
# Clean
all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['cleaned_combined_stores_4'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['cleaned_combined_stores_3'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_D_all_merge_4_40_above_involved_stores 2
already_defined_stores 454


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [76]:
# E - T1 >=50%
# merge 4: 3 stores - hold, max|min, nearest 10-28 miles


df_E_all_merge_3_40_above=all_paired_group_store_df[(all_paired_group_store_df['dist_to_nearest_store']>10) &\
                                           (all_paired_group_store_df['dist_to_nearest_store']<=28) &\
                                           ((all_paired_group_store_df['max_overlap_pctg']>=0.5)|(all_paired_group_store_df['min_overlap_pctg']>=0.5))]
df_E_all_merge_3_40_above['result_combined_stores']=df_E_all_merge_3_40_above['cleaned_combined_stores_3']

if len(df_E_all_merge_3_40_above)>0:
    list_df_E_all_merge_3_40_above_involved_stores=list(set(df_E_all_merge_3_40_above['result_combined_stores'].sum()))
else:
    list_df_E_all_merge_3_40_above_involved_stores=[]
print('list_df_E_all_merge_3_40_above_involved_stores',len(list_df_E_all_merge_3_40_above_involved_stores))
#### 
already_defined_stores=list(set(already_defined_stores+list_df_E_all_merge_3_40_above_involved_stores))
print("already_defined_stores",len(already_defined_stores))

######
# Clean
all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['cleaned_combined_stores_4'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['cleaned_combined_stores_3'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_E_all_merge_3_40_above_involved_stores 57
already_defined_stores 511


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# T2- >=35% & <50% overlap

In [77]:
# B - T2 [35%,50%)
# merge 1: 4 stores - hold, max&min, nearest without dist criteria

# nearest_always kept in the result
# whichever satisfied the cretearia, whichever should be included, max or min 

df_B_all_merge_4_35_40=all_paired_group_store_df[((all_paired_group_store_df['max_overlap_pctg']>=0.35) & (all_paired_group_store_df['max_overlap_pctg']<0.5)) &\
                                                 ((all_paired_group_store_df['min_overlap_pctg']>=0.35) & (all_paired_group_store_df['min_overlap_pctg']<0.5))]
df_B_all_merge_4_35_40['result_combined_stores']=df_B_all_merge_4_35_40['cleaned_combined_stores_4']

if len(df_B_all_merge_4_35_40)>0:
    list_df_B_all_merge_4_35_40_involved_stores=list(set(df_B_all_merge_4_35_40['result_combined_stores'].sum()))
else:
    list_df_B_all_merge_4_35_40_involved_stores=[]
    
print('list_df_B_all_merge_4_35_40_involved_stores',len(list_df_B_all_merge_4_35_40_involved_stores))
#### 
already_defined_stores=list(set(already_defined_stores+list_df_B_all_merge_4_35_40_involved_stores))
print("already_defined_stores",len(already_defined_stores))

######
# Clean
all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['cleaned_combined_stores_4'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['cleaned_combined_stores_3'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_B_all_merge_4_35_40_involved_stores 0
already_defined_stores 511


In [78]:
# C - T2 [35%,50%)
# merge 2: 3 stores - hold, max|min, nearest without dist criteria

df_C_all_merge_3_35_40=all_paired_group_store_df[((all_paired_group_store_df['max_overlap_pctg']>=0.35) & (all_paired_group_store_df['max_overlap_pctg']<0.5)) |\
                                                 ((all_paired_group_store_df['min_overlap_pctg']>=0.35) & (all_paired_group_store_df['min_overlap_pctg']<0.5))]

df_C_all_merge_3_35_40_1_Max=df_C_all_merge_3_35_40[(df_C_all_merge_3_35_40['max_overlap_pctg']>=0.35) & (df_C_all_merge_3_35_40['max_overlap_pctg']<0.5)]
df_C_all_merge_3_35_40_2_Min=df_C_all_merge_3_35_40[(df_C_all_merge_3_35_40['min_overlap_pctg']>=0.35) & (df_C_all_merge_3_35_40['min_overlap_pctg']<0.5)]

df_C_all_merge_3_35_40_1_Max['result_combined_stores']="["+"'"+df_C_all_merge_3_35_40_1_Max['store_id']+"'"+", "+"'"+df_C_all_merge_3_35_40_1_Max['nearest_store']+"'"+", "+"'"+df_C_all_merge_3_35_40_1_Max['max_paired_store']+"'"+"]"
df_C_all_merge_3_35_40_1_Max['result_combined_stores']=df_C_all_merge_3_35_40_1_Max['result_combined_stores'].apply(lambda x: list(set(eval(x))))

df_C_all_merge_3_35_40_2_Min['result_combined_stores']="["+"'"+df_C_all_merge_3_35_40_2_Min['store_id']+"'"+", "+"'"+df_C_all_merge_3_35_40_2_Min['nearest_store']+"'"+", "+"'"+df_C_all_merge_3_35_40_2_Min['min_paired_store']+"'"+"]"
df_C_all_merge_3_35_40_2_Min['result_combined_stores']=df_C_all_merge_3_35_40_2_Min['result_combined_stores'].apply(lambda x: list(set(eval(x))))

df_C_all_merge_3_35_40=df_C_all_merge_3_35_40_1_Max.append(df_C_all_merge_3_35_40_2_Min)

df_C_all_merge_3_35_40['result_combined_stores']=df_C_all_merge_3_35_40['result_combined_stores'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_C_all_merge_3_35_40_involved_stores=list(set(df_C_all_merge_3_35_40['result_combined_stores'].sum()))
print('list_df_C_all_merge_3_35_40_involved_stores',len(list_df_C_all_merge_3_35_40_involved_stores))

####
already_defined_stores=list(set(already_defined_stores+list_df_C_all_merge_3_35_40_involved_stores))
print("already_defined_stores",len(already_defined_stores))

######
# Clean
all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['cleaned_combined_stores_4'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['cleaned_combined_stores_3'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_C_all_merge_3_35_40_involved_stores 27
already_defined_stores 538


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

# T3- >=15% & <35% overlap

In [79]:
# B - T3 [15%,35%)
# merge 1: 4 stores - hold, max&min, nearest without dist criteria

# nearest_always kept in the result
# whichever satisfied the cretearia, whichever should be included, max or min 

df_B_all_merge_4_15_35=all_paired_group_store_df[((all_paired_group_store_df['max_overlap_pctg']>=0.15) & (all_paired_group_store_df['max_overlap_pctg']<0.35)) &\
                                                 ((all_paired_group_store_df['min_overlap_pctg']>=0.15) & (all_paired_group_store_df['min_overlap_pctg']<0.35))]
df_B_all_merge_4_15_35['result_combined_stores']=df_B_all_merge_4_15_35['cleaned_combined_stores_4']

if len(df_B_all_merge_4_15_35)>0:
    list_df_B_all_merge_4_15_35_involved_stores=list(set(df_B_all_merge_4_15_35['result_combined_stores'].sum()))
else:
    list_df_B_all_merge_4_15_35_involved_stores=[]
    
print('list_df_B_all_merge_4_15_35_involved_stores',len(list_df_B_all_merge_4_15_35_involved_stores))
#### 
already_defined_stores=list(set(already_defined_stores+list_df_B_all_merge_4_15_35_involved_stores))
print("already_defined_stores",len(already_defined_stores))

######
# Clean
all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['cleaned_combined_stores_4'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['cleaned_combined_stores_3'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_B_all_merge_4_15_35_involved_stores 1
already_defined_stores 539


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [80]:
# C - T3 [15%,35%)
# merge 2: 3 stores - hold, max|min, nearest without dist criteria

df_C_all_merge_3_15_35=all_paired_group_store_df[((all_paired_group_store_df['max_overlap_pctg']>=0.15) & (all_paired_group_store_df['max_overlap_pctg']<0.35)) |\
                                                 ((all_paired_group_store_df['min_overlap_pctg']>=0.15) & (all_paired_group_store_df['min_overlap_pctg']<0.35))]

df_C_all_merge_3_15_35_1_Max=df_C_all_merge_3_15_35[(df_C_all_merge_3_15_35['max_overlap_pctg']>=0.15) & (df_C_all_merge_3_15_35['max_overlap_pctg']<0.35)]
df_C_all_merge_3_15_35_2_Min=df_C_all_merge_3_15_35[(df_C_all_merge_3_15_35['min_overlap_pctg']>=0.15) & (df_C_all_merge_3_15_35['min_overlap_pctg']<0.35)]

df_C_all_merge_3_15_35_1_Max['result_combined_stores']="["+"'"+df_C_all_merge_3_15_35_1_Max['store_id']+"'"+", "+"'"+df_C_all_merge_3_15_35_1_Max['nearest_store']+"'"+", "+"'"+df_C_all_merge_3_15_35_1_Max['max_paired_store']+"'"+"]"
df_C_all_merge_3_15_35_1_Max['result_combined_stores']=df_C_all_merge_3_15_35_1_Max['result_combined_stores'].apply(lambda x: list(set(eval(x))))

df_C_all_merge_3_15_35_2_Min['result_combined_stores']="["+"'"+df_C_all_merge_3_15_35_2_Min['store_id']+"'"+", "+"'"+df_C_all_merge_3_15_35_2_Min['nearest_store']+"'"+", "+"'"+df_C_all_merge_3_15_35_2_Min['min_paired_store']+"'"+"]"
df_C_all_merge_3_15_35_2_Min['result_combined_stores']=df_C_all_merge_3_15_35_2_Min['result_combined_stores'].apply(lambda x: list(set(eval(x))))

df_C_all_merge_3_15_35=df_C_all_merge_3_15_35_1_Max.append(df_C_all_merge_3_15_35_2_Min)

df_C_all_merge_3_15_35['result_combined_stores']=df_C_all_merge_3_15_35['result_combined_stores'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_C_all_merge_3_15_35_involved_stores=list(set(df_C_all_merge_3_15_35['result_combined_stores'].sum()))
print('list_df_C_all_merge_3_15_35_involved_stores',len(list_df_C_all_merge_3_15_35_involved_stores))

####
already_defined_stores=list(set(already_defined_stores+list_df_C_all_merge_3_15_35_involved_stores))
print("already_defined_stores",len(already_defined_stores))

######
# Clean
all_paired_group_store_df['cleaned_combined_stores_4']=all_paired_group_store_df['cleaned_combined_stores_4'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))
all_paired_group_store_df['cleaned_combined_stores_3']=all_paired_group_store_df['cleaned_combined_stores_3'].apply(lambda x: identify_stores_already_asigned(x,already_defined_stores))


list_df_C_all_merge_3_15_35_involved_stores 13
already_defined_stores 552


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [81]:
remained_stores=all_paired_group_store_df[~all_paired_group_store_df['store_id'].isin(already_defined_stores)]

remained_stores['store_id'].nunique()

15

In [82]:
# Store list by group column

max_len_store=max(len(df_all_single_store_TA['store_id'].unique().tolist()),
                 len(list_df_B_all_merge_4_40_above_involved_stores),
                 len(list_df_C_all_merge_3_40_above_involved_stores),
                 len(list_df_D_all_merge_4_40_above_involved_stores),
                 len(list_df_E_all_merge_3_40_above_involved_stores),
                 len(list_df_B_all_merge_4_35_40_involved_stores),
                 len(list_df_C_all_merge_3_35_40_involved_stores),
                 len(list_df_B_all_merge_4_15_35_involved_stores),
                 len(list_df_C_all_merge_3_15_35_involved_stores),
                 )

df_stores_in_each_group=pd.DataFrame({"T1_A_single_store_TA":df_all_single_store_TA['store_id'].unique().tolist()+[np.nan]*(max_len_store-len(df_all_single_store_TA['store_id'].unique().tolist())),
                                      "T1_B_all_merge_4_40_above_involved_stores":list_df_B_all_merge_4_40_above_involved_stores+[np.nan]*(max_len_store-len(list_df_B_all_merge_4_40_above_involved_stores)),
                                      "T1_C_all_merge_3_40_above_involved_stores":list_df_C_all_merge_3_40_above_involved_stores+[np.nan]*(max_len_store-len(list_df_C_all_merge_3_40_above_involved_stores)),
                                      "T1_D_all_merge_4_40_above_involved_stores":list_df_D_all_merge_4_40_above_involved_stores+[np.nan]*(max_len_store-len(list_df_D_all_merge_4_40_above_involved_stores)),
                                      "T1_E_all_merge_3_40_above_involved_stores":list_df_E_all_merge_3_40_above_involved_stores+[np.nan]*(max_len_store-len(list_df_E_all_merge_3_40_above_involved_stores)),
                                      "T2_B_all_merge_4_35_40_involved_stores":list_df_B_all_merge_4_35_40_involved_stores+[np.nan]*(max_len_store-len(list_df_B_all_merge_4_35_40_involved_stores)),
                                      "T2_C_all_merge_3_35_40_involved_stores":list_df_C_all_merge_3_35_40_involved_stores+[np.nan]*(max_len_store-len(list_df_C_all_merge_3_35_40_involved_stores)),
                                      "T3_B_all_merge_4_15_35_involved_stores":list_df_B_all_merge_4_15_35_involved_stores+[np.nan]*(max_len_store-len(list_df_B_all_merge_4_15_35_involved_stores)),
                                      "T3_C_all_merge_3_15_35_involved_stores":list_df_C_all_merge_3_15_35_involved_stores+[np.nan]*(max_len_store-len(list_df_C_all_merge_3_15_35_involved_stores)),
                                     },index=[x for x in range(max_len_store)])
df_stores_in_each_group.shape

(393, 9)

In [83]:
df_stores_in_each_group

,T1_A_single_store_TA,T1_B_all_merge_4_40_above_involved_stores,T1_C_all_merge_3_40_above_involved_stores,T1_D_all_merge_4_40_above_involved_stores,T1_E_all_merge_3_40_above_involved_stores,T2_B_all_merge_4_35_40_involved_stores,T2_C_all_merge_3_35_40_involved_stores,T3_B_all_merge_4_15_35_involved_stores,T3_C_all_merge_3_15_35_involved_stores
0,1434,2654,2370,6950,3544,NaN,2250,7122,1132
1,6568,9250,2366,6923,6249,NaN,839,NaN,933
2,2639,8522,2771,NaN,2525,NaN,6260,NaN,333
3,2953,1420,280,NaN,1077,NaN,1128,NaN,2830
4,2928,8031,1239,NaN,2135,NaN,2439,NaN,1427
...,...,...,...,...,...,...,...,...,...
388,NaN,NaN,128,NaN,NaN,NaN,NaN,NaN,NaN
389,NaN,NaN,2394,NaN,NaN,NaN,NaN,NaN,NaN
390,NaN,NaN,4144,NaN,NaN,NaN,NaN,NaN,NaN
391,NaN,NaN,4863,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# See the stores defined with each condition

df_store_count=pd.DataFrame()

for col in df_stores_in_each_group.columns.tolist():
    list_col=df_stores_in_each_group[col].unique().tolist()
    list_col=[x for x in list_col if str(x)!="nan"]
    len_col=len(list_col)
    df=pd.DataFrame({"group":col,"store_count":len_col},index=[0])
    df_store_count=df_store_count.append(df)
df_store_count

,group,store_count
0,T1_A_single_store_TA,21
0,T1_B_all_merge_4_40_above_involved_stores,38
0,T1_C_all_merge_3_40_above_involved_stores,393
0,T1_D_all_merge_4_40_above_involved_stores,2
0,T1_E_all_merge_3_40_above_involved_stores,57
0,T2_B_all_merge_4_35_40_involved_stores,0
0,T2_C_all_merge_3_35_40_involved_stores,27
0,T3_B_all_merge_4_15_35_involved_stores,1
0,T3_C_all_merge_3_15_35_involved_stores,13


In [85]:
# Write out teh stores

writer=pd.ExcelWriter(output_folder+"GY_TA_combination_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_all_single_store_TA.to_excel(writer,"T1_df_all_single_store_TA",index=False)
df_B_all_merge_4_40_above.to_excel(writer,"T1_df_B_all_merge_4",index=False)
df_C_all_merge_3_40_above.to_excel(writer,"T1_df_C_all_merge_3",index=False)
df_D_all_merge_4_40_above.to_excel(writer,"T1_df_D_all_merge_4",index=False)
df_E_all_merge_3_40_above.to_excel(writer,"T1_df_E_all_merge_3",index=False)

df_B_all_merge_4_35_40.to_excel(writer,"T2_df_B_all_merge_4_15_35",index=False)
df_C_all_merge_3_35_40.to_excel(writer,"T2_df_C_all_merge_3_15_35",index=False)

df_B_all_merge_4_15_35.to_excel(writer,"T3_df_B_all_merge_4_15_35",index=False) # throw later
df_C_all_merge_3_15_35.to_excel(writer,"T3_df_C_all_merge_3_15_35",index=False) # throw later

df_store_count.to_excel(writer,"group_store_count",index=False)

df_stores_in_each_group.to_excel(writer,"defined_stores",index=False)

all_paired_group_store_df.to_excel(writer,"all_paired_group_store_df",index=False)
remained_stores.to_excel(writer,"remained_stores",index=False)
new_stores.to_excel(writer,"new_stores_without_P_S",index=False)
writer.save()

# Selected_N_stores_as_initial_aggregated_TA

In [86]:
DMA_nielsen=pd.read_excel(path_nielsen_zip,
                          dtype=str,skiprows=1,usecols=['CODE','NAME','NAME.1'])
DMA_nielsen=DMA_nielsen.rename(columns={"CODE":"zip_cd","NAME":"DMA","NAME.1":"CTY"})
DMA_nielsen=DMA_nielsen.drop_duplicates()
zip_DMA=DMA_nielsen.groupby("zip_cd")['DMA'].apply(set).to_frame().reset_index()
zip_CTY=DMA_nielsen.groupby("zip_cd")['CTY'].apply(set).to_frame().reset_index()
DMA_nielsen=pd.merge(zip_DMA,zip_CTY,on="zip_cd",how="left")

In [87]:
# Selected T1B - T2C
selected_groups=['T1_B_all_merge_4_40_above_involved_stores',
                 'T1_C_all_merge_3_40_above_involved_stores',
                 'T1_D_all_merge_4_40_above_involved_stores',
                 'T1_E_all_merge_3_40_above_involved_stores',
                 'T2_B_all_merge_4_35_40_involved_stores',
                 'T2_C_all_merge_3_35_40_involved_stores']

df_output_N_stores_selected=pd.DataFrame()

for group in selected_groups:
    store_list=df_stores_in_each_group[group].unique().tolist()
    store_list=[x for x in store_list if str(x)!="nan"]
    df=pd.DataFrame({"store_id":store_list},index=[group]*len(store_list)).reset_index().rename(columns={"index":"group"})
    df_output_N_stores_selected=df_output_N_stores_selected.append(df)
df_output_N_stores_selected.shape


(517, 2)

In [88]:
# Already defined

store_list=df_store_included_with_dates[['store_num','address_1','zip_cd','city','state','latitude','longitude']]
store_list=store_list.rename(columns={"store_num":"store_id"})
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_output_N_stores_selected=pd.merge(df_output_N_stores_selected,store_list,on="store_id",how="left")

df_output_N_stores_selected=pd.merge(df_output_N_stores_selected,DMA_nielsen,on="zip_cd",how="left")

df_output_N_stores_selected['DMA']=df_output_N_stores_selected['DMA'].astype(str)
df_output_N_stores_selected['CTY']=df_output_N_stores_selected['CTY'].astype(str)

df_output_N_stores_selected.head(2)

,group,store_id,address_1,zip_cd,city,state,latitude,longitude,DMA,CTY
0,T1_B_all_merge_4_40_above_involved_stores,2654,7991 US HIGHWAY 51 NORTH,38053,MILLINGTON,TN,35.34464,-89.907242,{'MEMPHIS'},"{'TIPTON', 'SHELBY'}"
1,T1_B_all_merge_4_40_above_involved_stores,9250,8922 ELK GROVE BLVD,95624,ELK GROVE,CA,38.40890,-121.370700,{'SACRAMNTO-STKTON-MODESTO'},{'SACRAMENTO'}


In [89]:
df_P_S_zips_by_store.head(2)

,store_id,all_P_zips,all_S_zips,40_miles_P_zips,40_miles_S_zips
0,1023,[15601],"[15644, 15642, 15650, 15231, 15401, 17103, 156...",[15601],"[15644, 15642, 15650, 15231, 15401, 15666, 156..."
1,1027,"[15227, 15122, 15120, 15207]","[15236, 15025, 15210, 15102, 15129, 15037, 151...","[15227, 15122, 15120, 15207]","[15236, 15025, 15210, 15102, 15129, 15037, 151..."


In [90]:
# Same above, add features to already selected stores

df_P_S_zips_by_store_copy=df_P_S_zips_by_store.copy()
df_P_S_zips_by_store_copy['All_P&S_zips']=df_P_S_zips_by_store_copy['all_P_zips']+df_P_S_zips_by_store_copy['all_S_zips']
df_P_S_zips_by_store_copy['All_P&S_zips_40_miles']=df_P_S_zips_by_store_copy['40_miles_P_zips']+df_P_S_zips_by_store_copy['40_miles_S_zips']
df_P_S_zips_by_store_copy=df_P_S_zips_by_store_copy[['store_id','all_P_zips','all_S_zips','All_P&S_zips','40_miles_P_zips','40_miles_S_zips','All_P&S_zips_40_miles']]

df_output_N_stores_selected=pd.merge(df_output_N_stores_selected,df_P_S_zips_by_store_copy,on="store_id",how="left")

df_output_N_stores_selected['nearest_store']=df_output_N_stores_selected['store_id'].apply(lambda x: find_nearest_store(x)[0])
df_output_N_stores_selected['dist_to_nearest_store']=df_output_N_stores_selected['store_id'].apply(lambda x: find_nearest_store(x)[1])

df_output_N_stores_selected=pd.merge(df_output_N_stores_selected,store_PS_40_near,on="nearest_store")
df_output_N_stores_selected['nearest_intersection_zips']=df_output_N_stores_selected.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['near_store_all_PS_40'])[0],axis=1)
df_output_N_stores_selected['nearest_pair_overlap_with_hold']=df_output_N_stores_selected.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['near_store_all_PS_40'])[1],axis=1)
df_output_N_stores_selected['nearest_pair_overlap_with_nearest']=df_output_N_stores_selected.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['near_store_all_PS_40'])[2],axis=1)



In [91]:
# Still, add columns to selected stores DF
df_output_N_stores_selected['min_overlap_pctg']=df_output_N_stores_selected['store_id'].apply(lambda x: find_overlaped_stores(x)[0])
df_output_N_stores_selected['min_overlap_store']=df_output_N_stores_selected['store_id'].apply(lambda x: find_overlaped_stores(x)[1])
df_output_N_stores_selected['max_overlap_pctg']=df_output_N_stores_selected['store_id'].apply(lambda x: find_overlaped_stores(x)[2])
df_output_N_stores_selected['max_overlap_store']=df_output_N_stores_selected['store_id'].apply(lambda x: find_overlaped_stores(x)[3])


df_output_N_stores_selected['min_overlap_store']=df_output_N_stores_selected['min_overlap_store'].apply(lambda x: eval(x))
df_output_N_stores_selected['max_overlap_store']=df_output_N_stores_selected['max_overlap_store'].apply(lambda x: eval(x))

df_output_N_stores_selected['min_paired_store']=df_output_N_stores_selected.apply(lambda x: remove_the_hold_store(x['min_overlap_store'],x['store_id']),axis=1)
df_output_N_stores_selected['max_paired_store']=df_output_N_stores_selected.apply(lambda x: remove_the_hold_store(x['max_overlap_store'],x['store_id']),axis=1)

df_output_N_stores_selected['min_overlap_store']=df_output_N_stores_selected['min_overlap_store'].astype(str)
df_output_N_stores_selected['max_overlap_store']=df_output_N_stores_selected['max_overlap_store'].astype(str)

df_output_N_stores_selected['dist_min_overlap_pairs']=df_output_N_stores_selected['min_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))
df_output_N_stores_selected['dist_max_overlap_pairs']=df_output_N_stores_selected['max_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))


df_output_N_stores_selected=pd.merge(df_output_N_stores_selected,store_PS_40_min,on="min_paired_store")
df_output_N_stores_selected=pd.merge(df_output_N_stores_selected,store_PS_40_max,on="max_paired_store")


df_output_N_stores_selected['max_intersection_zips']=df_output_N_stores_selected.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['max_store_all_PS_40'])[0],axis=1)
df_output_N_stores_selected['max_pair_overlap_with_hold']=df_output_N_stores_selected.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['max_store_all_PS_40'])[1],axis=1)
df_output_N_stores_selected['max_pair_overlap_with_max']=df_output_N_stores_selected.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['max_store_all_PS_40'])[2],axis=1)

df_output_N_stores_selected['min_intersection_zips']=df_output_N_stores_selected.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['min_store_all_PS_40'])[0],axis=1)
df_output_N_stores_selected['min_pair_overlap_with_hold']=df_output_N_stores_selected.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['min_store_all_PS_40'])[1],axis=1)
df_output_N_stores_selected['min_pair_overlap_with_min']=df_output_N_stores_selected.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['min_store_all_PS_40'])[2],axis=1)


In [92]:
# According to the defined conditions, summarize

df_output_N_stores_selected_with_result=pd.DataFrame()
for group,df_group in df_output_N_stores_selected.groupby("group"):
    print(group,df_group.shape[0])

    if (group in ['T1_B_all_merge_4_40_above_involved_stores','T2_B_all_merge_4_35_40_involved_stores']):
        df_1=df_group[df_group['min_overlap_pctg']>=0.35]
        df_2=df_group[df_group['min_overlap_pctg']<0.35]
        
        df_1['result_combined_store']=df_1[['store_id','nearest_store','max_paired_store','min_paired_store']].values.tolist()
        df_2['result_combined_store']=df_2[['store_id','nearest_store','max_paired_store']].values.tolist()
        
        df_group=df_1.append(df_2)
        
    elif (group in ['T1_C_all_merge_3_40_above_involved_stores','T1_D_all_merge_4_40_above_involved_stores','T1_E_all_merge_3_40_above_involved_stores']):
        df_group['result_combined_store']=df_group[['store_id','nearest_store','max_paired_store']].values.tolist()
        
    elif group=="T2_C_all_merge_3_35_40_involved_stores":
        df_group_max=df_group[(df_group['max_overlap_pctg']>=0.35) & (df_group['max_overlap_pctg']<0.5)]
        df_group_min=df_group[(df_group['min_overlap_pctg']>=0.35) & (df_group['min_overlap_pctg']<0.5)]
        list_max_group=df_group_max['store_id'].unique().tolist()
        list_min_group=df_group_min['store_id'].unique().tolist()
        list_both_group=list_max_group+list_min_group
        
        df_group_others=df_group[~df_group['store_id'].isin(list_both_group)]
        
        
        df_group_max['result_combined_store']=df_group_max[['store_id','nearest_store','max_paired_store']].values.tolist()
        df_group_min['result_combined_store']=df_group_min[['store_id','nearest_store','min_paired_store']].values.tolist()
        
        df_group=df_group_max.append(df_group_min).append(df_group_others)
        
    print(group,len(df_group))
    df_output_N_stores_selected_with_result=df_output_N_stores_selected_with_result.append(df_group)
df_output_N_stores_selected_with_result['result_combined_store']=df_output_N_stores_selected_with_result['result_combined_store'].fillna("[]")
df_output_N_stores_selected_with_result['result_combined_store']=df_output_N_stores_selected_with_result['result_combined_store'].astype(str)
df_output_N_stores_selected_with_result['result_combined_store']=df_output_N_stores_selected_with_result['result_combined_store'].apply(lambda x: sorted(list(set(eval(x)))))

T1_B_all_merge_4_40_above_involved_stores 38
T1_B_all_merge_4_40_above_involved_stores 38
T1_C_all_merge_3_40_above_involved_stores 393
T1_C_all_merge_3_40_above_involved_stores 393
T1_D_all_merge_4_40_above_involved_stores 2
T1_D_all_merge_4_40_above_involved_stores 2
T1_E_all_merge_3_40_above_involved_stores 57
T1_E_all_merge_3_40_above_involved_stores 57
T2_C_all_merge_3_35_40_involved_stores 27
T2_C_all_merge_3_35_40_involved_stores 27


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [93]:
print(df_output_N_stores_selected_with_result.shape,df_output_N_stores_selected_with_result['store_id'].nunique())
df_output_N_stores_selected_with_result.head(4)

(517, 39) 517


,group,store_id,address_1,zip_cd,city,state,latitude,longitude,DMA,CTY,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,near_store_all_PS_40,nearest_intersection_zips,nearest_pair_overlap_with_hold,nearest_pair_overlap_with_nearest,min_overlap_pctg,min_overlap_store,max_overlap_pctg,max_overlap_store,min_paired_store,max_paired_store,dist_min_overlap_pairs,dist_max_overlap_pairs,min_store_all_PS_40,max_store_all_PS_40,max_intersection_zips,max_pair_overlap_with_hold,max_pair_overlap_with_max,min_intersection_zips,min_pair_overlap_with_hold,min_pair_overlap_with_min,result_combined_store
1,T1_B_all_merge_4_40_above_involved_stores,2620,1200 UNION AVENUE,38104,MEMPHIS,TN,35.137920,-90.021379,{'MEMPHIS'},{'SHELBY'},"[38104, 38106, 38107, 38109, 38116, 38127, 381...","[38103, 38112, 38117, 38105, 38134, 38122, 381...","[38104, 38106, 38107, 38109, 38116, 38127, 381...","[38104, 38106, 38107, 38109, 38116, 38127, 381...","[38103, 38112, 38117, 38105, 38134, 38122, 381...","[38104, 38106, 38107, 38109, 38116, 38127, 381...",2650,9.406132,"[38028, 72376, 38122, 38049, 38174, 72348, 380...","[38028, 72376, 38049, 38122, 38174, 72348, 380...",0.747748,0.798077,0.613208,"['2620', '2654']",0.759259,"['2620', '2621']",2654,2621,15.668054,11.716155,"[38028, 38049, 38122, 72376, 38060, 38107, 380...","[38028, 38685, 38049, 38122, 72376, 38174, 723...","[38028, 38049, 72376, 38122, 38174, 72348, 380...",0.792793,0.880000,"[38028, 38184, 38058, 38054, 38116, 38049, 723...",0.639640,0.876543,"[2620, 2621, 2650, 2654]"
3,T1_B_all_merge_4_40_above_involved_stores,2650,1625 NORTH MISSOURI STREET,72301,WEST MEMPHIS,AR,35.164495,-90.184676,{'MEMPHIS'},{'CRITTENDEN'},[72301],"[72364, 72303, 72331, 72376, 72327, 72348, 723...","[72301, 72364, 72303, 72331, 72376, 72327, 723...",[72301],"[72364, 72303, 72331, 72376, 72327, 72348, 723...","[72301, 72364, 72303, 72331, 72376, 72327, 723...",2620,9.406132,"[38028, 72376, 38122, 38049, 38174, 38131, 723...","[38028, 38049, 72376, 38122, 38174, 72348, 380...",0.798077,0.747748,0.513761,"['2650', '2654']",0.627119,"['2620', '2650']",2654,2620,19.998649,9.406132,"[38028, 38049, 38122, 72376, 38060, 38107, 380...","[38028, 72376, 38122, 38049, 38174, 38131, 723...","[38028, 38049, 72376, 38122, 38174, 72348, 380...",0.798077,0.747748,"[38028, 38058, 38116, 38049, 72376, 38122, 380...",0.586538,0.753086,"[2620, 2650, 2654]"
14,T1_B_all_merge_4_40_above_involved_stores,8522,7986 GREENBACK LANE,95610,CITRUS HEIGHTS,CA,38.677342,-121.269329,{'SACRAMNTO-STKTON-MODESTO'},{'SACRAMENTO'},"[95610, 95621]","[95628, 95662, 95608, 95843, 95670, 95842, 956...","[95610, 95621, 95628, 95662, 95608, 95843, 956...","[95610, 95621]","[95628, 95662, 95608, 95843, 95670, 95842, 956...","[95610, 95621, 95628, 95662, 95608, 95843, 956...",9251,7.696455,"[95993, 95616, 95656, 95776, 95645, 95757, 956...","[95993, 95616, 95776, 95757, 95626, 95670, 958...",0.759690,0.796748,0.529032,"['8522', '9250']",0.720588,"['8521', '8522']",9250,8521,19.339665,10.018856,"[95616, 95242, 95210, 94513, 95759, 95690, 958...","[95993, 95616, 95242, 95912, 95210, 94513, 957...","[95993, 95616, 95242, 95210, 95690, 95832, 957...",0.906977,0.754839,"[95616, 95242, 95210, 95690, 95832, 95776, 957...",0.782946,0.682432,"[8521, 8522, 9250, 9251]"
16,T1_B_all_merge_4_40_above_involved_stores,9251,5470 PACIFIC STREET,95677,ROCKLIN,CA,38.786333,-121.239841,{'SACRAMNTO-STKTON-MODESTO'},{'PLACER'},"[95677, 95765]","[95650, 95648, 95747, 95746, 95678, 95661, 956...","[95677, 95765, 95650, 95648, 95747, 95746, 956...","[95677, 95765]","[95650, 95648, 95747, 95746, 95678, 95661, 956...","[95677, 95765, 95650, 95648, 95747, 95746, 956...",8533,3.104836,"[95993, 95616, 95242, 95832, 95776, 95757, 956...","[95993, 95616, 95776, 95757, 95626, 95670, 958...",0.869919,0.849206,0.439024,"['9250', '9251']",0.798387,"['8533', '9251']",9250,8533,27.018548,3.104836,"[95616, 95242, 95210, 94513, 9575

In [94]:
# Rerange

col_list_2=col_list.copy()
col_list_2.remove("dist_group")

col_list_2=col_list_2[:6]+['CTY','DMA']+col_list_2[6:]
df_output_N_stores_selected_with_result=df_output_N_stores_selected_with_result[["group"]+col_list_2+["result_combined_store"]]

In [95]:
# Write out T1b and T2c
df_output_N_stores_selected_with_result=df_output_N_stores_selected_with_result.sort_values(["dist_to_nearest_store","state","CTY","city"])

writer=pd.ExcelWriter(output_folder+"sorted_N_stores_T1B_T2C_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_N_stores_selected_with_result.to_excel(writer,"sorted_N_stores",index=False)
writer.save()


In [97]:
# Name the TA number

df_data=df_output_N_stores_selected_with_result[['store_id','result_combined_store']]
df_ta_store_result_pairs=pd.DataFrame(columns=['TA','store_id'])
df_data=df_data.reset_index()
del df_data['index']

ta=0
for i,row in df_data.iterrows():
    location_hold=row['store_id']
    location_associatd=row['result_combined_store']
    intersection_existing=set(location_associatd).intersection(set(df_ta_store_result_pairs['store_id'].tolist()))
    
    if len(intersection_existing)==0:
        ta+=1
        df=pd.DataFrame({"TA":[ta]*len(location_associatd),"store_id":location_associatd},index=[ta]*len(location_associatd))
        df_ta_store_result_pairs=df_ta_store_result_pairs.append(df)
    else:
        TA_list_overlapped=df_ta_store_result_pairs[df_ta_store_result_pairs['store_id'].isin(intersection_existing)]['TA'].unique().tolist()
        grouped_ta=df_ta_store_result_pairs[df_ta_store_result_pairs['TA'].isin(TA_list_overlapped)]
        grouped_ta_others=df_ta_store_result_pairs[~df_ta_store_result_pairs['TA'].isin(TA_list_overlapped)]
        
        grouped_ta['TA']=grouped_ta['TA'].min()
        
        new_stores_added=[x for x in location_associatd if x not in grouped_ta['store_id'].tolist()]
        df=pd.DataFrame({"TA":[grouped_ta['TA'].min()]*len(new_stores_added),"store_id":new_stores_added},index=[grouped_ta['TA'].min()]*len(new_stores_added))
        grouped_ta=grouped_ta.append(df)
        df_ta_store_result_pairs=grouped_ta_others.append(grouped_ta)
        

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [98]:
df_ta_store_result_pairs=df_ta_store_result_pairs[df_ta_store_result_pairs['store_id'].isin(df_output_N_stores_selected_with_result['store_id'].tolist())]
print(df_ta_store_result_pairs.shape,df_ta_store_result_pairs['store_id'].nunique())
df_ta_store_result_pairs.head(3)

(517, 2) 517


,TA,store_id
7.0,7.0,2723
7.0,7.0,2729
10.0,10.0,4321


In [99]:
df_output_N_stores_selected_with_result=pd.merge(df_output_N_stores_selected_with_result,df_ta_store_result_pairs,on="store_id",how="left")

summary_df_N_stores=df_output_N_stores_selected_with_result.groupby("TA")['store_id'].count().to_frame().reset_index().sort_values("store_id",ascending=False)


In [100]:
df_output_N_stores_selected_with_result=df_output_N_stores_selected_with_result[['TA']+[x for x in df_output_N_stores_selected_with_result.columns.tolist() if x!="TA"]]
df_output_N_stores_selected_with_result=df_output_N_stores_selected_with_result.sort_values(['TA','state','CTY'])



In [101]:
df_rename_TA=df_output_N_stores_selected_with_result[['TA']].drop_duplicates()
df_rename_TA['new_name']=[x+1 for x in range(len(df_rename_TA))]
dict_rename_TA=df_rename_TA.set_index("TA").to_dict()["new_name"]


df_output_N_stores_selected_with_result['TA']=df_output_N_stores_selected_with_result['TA'].apply(lambda x: dict_rename_TA[x])



In [102]:
# Write out initial TA

writer=pd.ExcelWriter(output_folder+"initial_TA_output.xlsx",engine="xlsxwriter")

df_output_N_stores_selected_with_result.to_excel(writer,"with_TA",index=False)
summary_df_N_stores.to_excel(writer,"summary",index=False)

writer.save()

In [106]:
df_output_N_stores_selected_with_result[df_output_N_stores_selected_with_result['TA']==3].shape

(21, 40)

In [107]:
# Append columns of summarized into TA level

df_output_N_TA_zips=df_output_N_stores_selected_with_result.groupby("TA")['All_P&S_zips_40_miles','All_P&S_zips'].sum().reset_index()

df_output_N_TA_zips=df_output_N_TA_zips.rename(columns={"All_P&S_zips_40_miles":"zip_cd_40","All_P&S_zips":"zip_cd_all","store_id":"store_list"})
df_output_N_TA_zips['zip_cd_40']=df_output_N_TA_zips['zip_cd_40'].apply(lambda x: list(set(x)))
df_output_N_TA_zips['zip_cd_all']=df_output_N_TA_zips['zip_cd_all'].apply(lambda x: list(set(x)))

df_output_N_TA_zips_2=df_output_N_stores_selected_with_result.groupby("TA")['store_id'].apply(list).reset_index()
df_output_N_TA_zips_2=df_output_N_TA_zips_2.rename(columns={"store_id":"store_list"})
df_output_N_TA_zips_2['store_list']=df_output_N_TA_zips_2['store_list'].apply(lambda x: list(set(x)))

df_output_N_TA_zips=pd.merge(df_output_N_TA_zips,df_output_N_TA_zips_2,on="TA")
print(df_output_N_TA_zips.shape)
df_output_N_TA_zips.head(3)

(118, 4)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TA,zip_cd_40,zip_cd_all,store_list
0,1,"[95003, 95039, 95148, 94402, 94035, 94122, 950...","[92563, 91304, 94402, 95210, 37681, 93944, 954...","[8724, 9246, 4861, 1290]"
1,2,"[30253, 31064, 30571, 30350, 30107, 30311, 301...","[91304, 30107, 48224, 25901, 48176, 37322, 295...","[2126, 2130, 2121, 2141, 2136, 3610, 2128]"
2,3,"[46375, 60526, 60015, 60017, 60020, 60155, 606...","[15314, 55347, 95670, 34608, 60030, 60456, 610...","[6125, 1275, 1282, 1267, 1271, 3602, 1265, 128..."


In [108]:
a=df_output_N_TA_zips['store_list'].sum()
len(set(a))

517

# Single Store TA¶

In [109]:
# Since no new stores, skip here
new_stores=new_stores.reset_index()

df_new_store_zips=pd.DataFrame()

del new_stores['index']

for ind,row in new_stores.iterrows():
    store_num=row['store_id']
    store_center=[row['latitude'],row['longitude']]
    
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],unit="mi")
        
        if dist<=10:

            df=pd.DataFrame({"store_id":store_num,"zips_in_10":zip_cd,"dist_miles":dist},index=[store_num])
            df_new_store_zips=df_new_store_zips.append(df)
        
df_new_store_zips.shape

# No new stores!!!

(0, 0)

In [110]:
# The stores that have no overlap with others are assigned as a single store TA

df_sing_store_zips_wide=df_all_single_store_TA[['store_id','All_P&S_zips_40_miles','All_P&S_zips']]

df_sing_store_zips_wide=df_sing_store_zips_wide.sort_values("store_id")
df_sing_store_zips_wide=df_sing_store_zips_wide.rename(columns={"All_P&S_zips_40_miles":"zip_cd_40",
                                                                "All_P&S_zips":"zip_cd_all",
                                                                "store_id":"store_list"})
df_sing_store_zips_wide['store_list']=df_sing_store_zips_wide['store_list'].apply(lambda x: [x])
df_sing_store_zips_wide['TA']=[x+1+df_output_N_TA_zips['TA'].max() for x in range(len(df_sing_store_zips_wide))]

all_defined_TA=df_output_N_TA_zips.append(df_sing_store_zips_wide)
all_defined_stores=all_defined_TA['store_list'].sum()
print(len(all_defined_stores))

all_defined_TA=all_defined_TA.reset_index()
del all_defined_TA['index']

all_defined_TA.shape

538


(139, 4)

In [111]:
df_output_N_TA_zips

,TA,zip_cd_40,zip_cd_all,store_list
0,1,"[95003, 95039, 95148, 94402, 94035, 94122, 950...","[92563, 91304, 94402, 95210, 37681, 93944, 954...","[8724, 9246, 4861, 1290]"
1,2,"[30253, 31064, 30571, 30350, 30107, 30311, 301...","[91304, 30107, 48224, 25901, 48176, 37322, 295...","[2126, 2130, 2121, 2141, 2136, 3610, 2128]"
2,3,"[46375, 60526, 60015, 60017, 60020, 60155, 606...","[15314, 55347, 95670, 34608, 60030, 60456, 610...","[6125, 1275, 1282, 1267, 1271, 3602, 1265, 128..."
3,4,"[95003, 95039, 95148, 94402, 94035, 94122, 946...","[92563, 94402, 95210, 93944, 92223, 30152, 553...","[8722, 8773, 8766]"
4,5,"[80235, 80107, 80011, 80534, 80452, 80238, 806...","[76513, 92563, 50518, 12169, 30107, 29550, 922...","[8120, 8154, 8121, 8165, 8179, 8127, 8124, 816..."
...,...,...,...,...
113,114,"[78594, 78538, 78592, 78597, 78584, 78563, 785...","[95210, 49637, 55127, 77023, 78840, 53094, 543...","[4721, 4737, 4752]"
114,115,"[41063, 45237, 47030, 41092, 45212, 45203, 452...","[46375, 20695, 84093, 30703, 30253, 66207, 452...","[6225, 6249]"
115,116,"[94510, 94402, 94604, 95148, 94122, 94613, 949...","[20695, 92563, 91304, 94402, 95210, 95453, 922...","[8728, 9243]"
116,117,"[32568, 36578, 36542, 32503, 36505, 36590, 325...","[30120, 32428, 32025, 35801, 30152, 97801, 727...","[2222, 2293]"


In [112]:
# Remaining stores

df_remain_stores=store_list[['store_id']].drop_duplicates()

df_remain_stores=df_remain_stores[~df_remain_stores['store_id'].isin(all_defined_stores)]
df_remain_stores.shape

(29, 1)

In [113]:
# All of the rest are how to put the remaining stores into existing defined TAs

df_remain_stores=pd.merge(df_remain_stores,store_list,on="store_id",how="left")
df_remain_stores=pd.merge(df_remain_stores,DMA_nielsen,on="zip_cd",how="left")

df_remain_stores['DMA']=df_remain_stores['DMA'].astype(str)
df_remain_stores['CTY']=df_remain_stores['CTY'].astype(str)

In [114]:
# Focus on remaining only

df_remain_stores=pd.merge(df_remain_stores,df_P_S_zips_by_store_copy,on="store_id",how="inner")

df_remain_stores['nearest_store']=df_remain_stores['store_id'].apply(lambda x: find_nearest_store(x)[0])
df_remain_stores['dist_to_nearest_store']=df_remain_stores['store_id'].apply(lambda x: find_nearest_store(x)[1])

df_remain_stores=pd.merge(df_remain_stores,store_PS_40_near,on="nearest_store")

df_remain_stores['nearest_intersection_zips']=df_remain_stores.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['near_store_all_PS_40'])[0],axis=1)
df_remain_stores['nearest_pair_overlap_with_hold']=df_remain_stores.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['near_store_all_PS_40'])[1],axis=1)
df_remain_stores['nearest_pair_overlap_with_nearest']=df_remain_stores.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['near_store_all_PS_40'])[2],axis=1)


In [115]:
# Same as above steps, prepare the DF with zips and overlap cols
df_remain_stores['min_overlap_pctg']=df_remain_stores['store_id'].apply(lambda x: find_overlaped_stores(x)[0])
df_remain_stores['min_overlap_store']=df_remain_stores['store_id'].apply(lambda x: find_overlaped_stores(x)[1])
df_remain_stores['max_overlap_pctg']=df_remain_stores['store_id'].apply(lambda x: find_overlaped_stores(x)[2])
df_remain_stores['max_overlap_store']=df_remain_stores['store_id'].apply(lambda x: find_overlaped_stores(x)[3])


df_remain_stores['min_overlap_store']=df_remain_stores['min_overlap_store'].apply(lambda x: eval(x))
df_remain_stores['max_overlap_store']=df_remain_stores['max_overlap_store'].apply(lambda x: eval(x))

df_remain_stores['min_paired_store']=df_remain_stores.apply(lambda x: remove_the_hold_store(x['min_overlap_store'],x['store_id']),axis=1)
df_remain_stores['max_paired_store']=df_remain_stores.apply(lambda x: remove_the_hold_store(x['max_overlap_store'],x['store_id']),axis=1)

df_remain_stores['min_overlap_store']=df_remain_stores['min_overlap_store'].astype(str)
df_remain_stores['max_overlap_store']=df_remain_stores['max_overlap_store'].astype(str)

df_remain_stores['dist_min_overlap_pairs']=df_remain_stores['min_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))
df_remain_stores['dist_max_overlap_pairs']=df_remain_stores['max_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))


df_remain_stores=pd.merge(df_remain_stores,store_PS_40_min,on="min_paired_store")
df_remain_stores=pd.merge(df_remain_stores,store_PS_40_max,on="max_paired_store")


df_remain_stores['max_intersection_zips']=df_remain_stores.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['max_store_all_PS_40'])[0],axis=1)
df_remain_stores['max_pair_overlap_with_hold']=df_remain_stores.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['max_store_all_PS_40'])[1],axis=1)
df_remain_stores['max_pair_overlap_with_max']=df_remain_stores.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['max_store_all_PS_40'])[2],axis=1)

df_remain_stores['min_intersection_zips']=df_remain_stores.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['min_store_all_PS_40'])[0],axis=1)
df_remain_stores['min_pair_overlap_with_hold']=df_remain_stores.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['min_store_all_PS_40'])[1],axis=1)
df_remain_stores['min_pair_overlap_with_min']=df_remain_stores.apply(lambda x: intersection_2_list(x['All_P&S_zips_40_miles'],x['min_store_all_PS_40'])[2],axis=1)


In [108]:
# Because we don't have new store df
# df_remain_stores=df_remain_stores.append(df_new_store_zips_wide_copy)

In [116]:
# Fuction to see which existing TA can cover the best of each remaining store
def find_max_covered_TA(zip_list_store):
    max_intersect_count=0
    max_intersect_list=[]
    intersect_TA=0
    for ind,row in all_defined_TA.iterrows():
        ta_40_zips=row['zip_cd_40']
        intersection_zips_P_S_40=list(set(zip_list_store).intersection(set(ta_40_zips)))
        intersection_zips_P_S_40_len=len(intersection_zips_P_S_40)
        if intersection_zips_P_S_40_len>max_intersect_count:
            max_intersect_count=intersection_zips_P_S_40_len
            max_intersect_list=intersection_zips_P_S_40
            intersect_TA=row['TA']
    max_intersect_pctc=max_intersect_count/len(zip_list_store)
            
    return intersect_TA,max_intersect_list,max_intersect_count,max_intersect_pctc

In [117]:
# Apply above func

df_remain_stores['Max_overlap_TA']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[0])
df_remain_stores['Overlap_zips_with_TA_40']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[1])
df_remain_stores['Overlap_zips_count']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[2])
df_remain_stores['Overlap_zips_Pctg']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[3])


In [119]:
df_remain_stores.head(2)

,store_id,address_1,zip_cd,city,state,latitude,longitude,DMA,CTY,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,near_store_all_PS_40,nearest_intersection_zips,nearest_pair_overlap_with_hold,nearest_pair_overlap_with_nearest,min_overlap_pctg,min_overlap_store,max_overlap_pctg,max_overlap_store,min_paired_store,max_paired_store,dist_min_overlap_pairs,dist_max_overlap_pairs,min_store_all_PS_40,max_store_all_PS_40,max_intersection_zips,max_pair_overlap_with_hold,max_pair_overlap_with_max,min_intersection_zips,min_pair_overlap_with_hold,min_pair_overlap_with_min,Max_overlap_TA,Overlap_zips_with_TA_40,Overlap_zips_count,Overlap_zips_Pctg
0,2264,297 WEST MEIGHAN BLVD,35901,GADSDEN,AL,34.015013,-86.000127,{'BIRMINGHAM (ANN AND TUSC)'},"{'ETOWAH', 'CHEROKEE'}","[35901, 35904]","[35903, 35906, 35907, 35905, 35954, 36265, 362...","[35901, 35904, 35903, 35906, 35907, 35905, 359...","[35901, 35904]","[35903, 35906, 35907, 35905, 35954, 36265, 362...","[35901, 35904, 35903, 35906, 35907, 35905, 359...",2124,48.283653,"[30703, 30107, 30101, 30747, 30120, 35904, 301...","[30125, 35988, 35973, 35962, 30124, 35959, 359...",0.191011,0.175258,0.021739,"['2146', '2264']",0.076923,"['2124', '2264']",2146,2124,60.941831,48.283653,"[30185, 30311, 30101, 30308, 30319, 30120, 362...","[30703, 30107, 30101, 30747, 30120, 35904, 301...","[30125, 35988, 35973, 35962, 30124, 35959, 359...",0.191011,0.175258,"[30125, 36206, 36277, 36265, 36201, 36207, 362...",0.134831,0.089552,37,"[35121, 35180, 35235, 35079, 35096, 35215, 351...",31,0.348315
1,2647,603 NORTH ARKANSAS AVENUE,72801,RUSSELLVILLE,AR,35.282311,-93.133585,{'LITTLE ROCK-PINE BLUFF'},{'POPE'},"[72802, 72837]","[72801, 72834, 72830, 72823, 72858, 72811, 728...","[72802, 72837, 72801, 72834, 72830, 72823, 728...","[72802, 72837]","[72801, 72834, 72830, 72823, 72858, 72811, 728...","[72802, 72837, 72801, 72834, 72830, 72823, 728...",2626,61.165292,"[72023, 72150, 72206, 72202, 72078, 72142, 721...","[71909, 72034, 72135, 72032, 72070, 72110, 720...",0.161765,0.115789,0.031250,"['2647', '2677']",0.039062,"['2626', '2647']",2677,2626,61.209486,61.165292,"[72023, 72150, 72206, 72202, 72142, 71949, 721...","[72023, 72150, 72206, 72202, 72078, 72142, 721...","[71909, 72034, 72135, 72032, 72070, 72110, 720...",0.161765,0.115789,"[71909, 71949, 72135, 72034, 72032, 72126, 720...",0.161765,0.117021,84,"[71909, 71949, 72135, 72034, 72032, 72070, 721...",13,0.191176


In [120]:
# another df to be used

del df_all_single_store_TA['dist_group']
df_sing_store_zips_wide_copy=df_sing_store_zips_wide.copy()
df_sing_store_zips_wide_copy=df_sing_store_zips_wide_copy[['store_list','TA']]
df_sing_store_zips_wide_copy['store_id']=df_sing_store_zips_wide_copy['store_list'].apply(lambda x: x[0])
df_sing_store_zips_wide_copy=df_sing_store_zips_wide_copy[['store_id',"TA"]]


In [121]:
df_all_single_store_TA.head(2)

,store_id,address_1,city,state,zip_cd,latitude,longitude,all_P_zips,all_S_zips,All_P&S_zips,40_miles_P_zips,40_miles_S_zips,All_P&S_zips_40_miles,nearest_store,dist_to_nearest_store,near_store_all_PS_40,nearest_intersection_zips,nearest_pair_overlap_with_hold,nearest_pair_overlap_with_nearest,max_overlap_store,max_overlap_pctg,dist_max_overlap_pairs,max_paired_store,max_store_all_PS_40,max_intersection_zips,max_pair_overlap_with_hold,max_pair_overlap_with_max,min_overlap_store,min_overlap_pctg,dist_min_overlap_pairs,min_paired_store,min_store_all_PS_40,min_intersection_zips,min_pair_overlap_with_hold,min_pair_overlap_with_min
415,1434,4119 AVENUE OF THE CITIES,MOLINE,IL,61265,41.490201,-90.479416,"[61265, 61201]","[61244, 61275, 61282, 61264, 61241, 61240, 612...","[61265, 61201, 61244, 61275, 61282, 61264, 612...","[61265, 61201]","[61244, 61275, 61282, 61264, 61241, 61240, 612...","[61265, 61201, 61244, 61275, 61282, 61264, 612...",1425,66.024494,"[61401, 61423, 61730, 62682, 61491, 61335, 626...","[61401, 61443, 61490, 61458, 61428, 61361]",0.075949,0.053097,"[1425, 1434]",0.006173,66.024494,1425,"[61401, 61423, 61730, 62682, 61491, 61335, 626...","[61401, 61443, 61490, 61458, 61428, 61361]",0.075949,0.053097,"[1425, 1434]",0.006173,66.024494,1425,"[61401, 61423, 61730, 62682, 61491, 61335, 626...","[61401, 61443, 61490, 61458, 61428, 61361]",0.075949,0.053097
452,6568,5029 SOUTH WESTNEDGE AVENUE,PORTAGE,MI,49002,42.244737,-85.589549,"[49001, 49009, 49008, 49002, 49048]","[49024, 49003, 49006, 49097, 49079, 49007, 490...","[49001, 49009, 49008, 49002, 49048, 49024, 490...","[49001, 49009, 49008, 49002, 49048]","[49024, 49003, 49006, 49097, 49079, 49007, 490...","[49001, 49009, 49008, 49002, 49048, 49024, 490...",6561,60.049367,"[49202, 49251, 49230, 49234, 49285, 49015, 492...","[49316, 49015, 49302, 49014, 49331, 49546, 490...",0.162602,0.144928,"[6561, 6568]",0.033898,60.049367,6561,"[49202, 49251, 49230, 49234, 49285, 49015, 492...","[49316, 49015, 49302, 49014, 49331, 49546, 490...",0.162602,0.144928,"[6561, 6568]",0.033898,60.049367,6561,"[49202, 49251, 49230, 49234, 49285, 49015, 492...","[49316, 49015, 49302, 49014, 49331, 49546, 490...",0.162602,0.144928


In [122]:
df_sing_store_zips_wide_copy.head(2) # single

,store_id,TA
415,1434,119
0,2638,120


In [123]:
# Add TA num
df_all_single_store_TA=pd.merge(df_all_single_store_TA,df_sing_store_zips_wide_copy,on="store_id",how="left")
df_all_single_store_TA=df_all_single_store_TA[["TA"]+[x for x in df_all_single_store_TA.columns.tolist() if x!="TA"]]

In [124]:
# Fill the gap 1-n to see which avaible, since some TA may merged together above
df_all_single_store_TA_cols_1=[x for x in df_output_N_stores_selected_with_result.columns.tolist() if x in df_all_single_store_TA.columns.tolist()]
df_all_single_store_TA_cols_2=[x for x in df_all_single_store_TA.columns.tolist() if x not in df_all_single_store_TA_cols_1]
df_all_single_store_TA=df_all_single_store_TA[df_all_single_store_TA_cols_1+df_all_single_store_TA_cols_2]

In [125]:

df_remain_stores_cols_1=[x for x in df_output_N_stores_selected_with_result.columns.tolist() if x in df_remain_stores.columns.tolist()]
df_remain_stores_cols_2=[x for x in df_remain_stores.columns.tolist() if x not in df_remain_stores_cols_1]
df_remain_stores=df_remain_stores[df_remain_stores_cols_1+df_remain_stores_cols_2]

In [126]:
# Write out

writer=pd.ExcelWriter(output_folder+"GY_remaining_stores_and_defined_TA_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_remain_stores.to_excel(writer,"df_remain_stores",index=False)
all_defined_TA.to_excel(writer,"all_defined_TA",index=False)
df_output_N_stores_selected_with_result.to_excel(writer,"TA_by_store_multiple",index=False)
df_all_single_store_TA.to_excel(writer,"TA_by_store_single",index=False)
writer.save()


In [118]:
all_defined_TA.head(4)

,TA,zip_cd_40,zip_cd_all,store_list
0,1,"[95121, 95151, 94107, 94549, 95128, 94122, 953...","[95959, 92101, 97005, 88061, 95661, 78664, 931...","[9246, 8724, 1290, 4861]"
1,2,"[30349, 30236, 30094, 30641, 30630, 30601, 303...","[66018, 32459, 48304, 76021, 48176, 27804, 330...","[2141, 2126, 2121, 2130, 3610, 2136, 2128]"
2,3,"[61012, 60607, 60436, 60115, 60016, 60161, 531...","[85742, 46816, 92101, 48176, 50677, 46182, 461...","[1265, 1282, 6120, 1283, 1284, 1271, 6133, 127..."
3,4,"[95121, 95151, 94107, 94549, 95128, 94122, 943...","[95959, 92101, 19103, 98007, 96122, 76135, 956...","[8773, 8722, 8766]"


In [127]:
print(datetime.datetime.now())

2020-12-01 14:41:38.043168


In [129]:
# We have the DF for the remaining stores, go to allocate them
df_remain_stores.shape

(29, 41)

# Iter the balance¶

In [130]:
# Steps to lower the overlap pctg threshold to put the remaining into existing

In [131]:
# Iter 1 - >=40% 

df_remain_stores_Iter1_40=df_remain_stores[df_remain_stores['Overlap_zips_Pctg']>=0.4]
df_remain_stores_Iter1_40=df_remain_stores_Iter1_40[['store_id','Max_overlap_TA','All_P&S_zips','All_P&S_zips_40_miles']]
df_remain_stores_Iter1_40=df_remain_stores_Iter1_40.rename(columns={"Max_overlap_TA":"TA"})
df_remain_stores_Iter1_40['store_id']=df_remain_stores_Iter1_40['store_id'].apply(lambda x: [x])
df_remain_stores_Iter1_40=df_remain_stores_Iter1_40.groupby("TA")['store_id','All_P&S_zips','All_P&S_zips_40_miles'].sum().reset_index()
for col in ['store_id','All_P&S_zips','All_P&S_zips_40_miles']:
    df_remain_stores_Iter1_40[col]=df_remain_stores_Iter1_40[col].apply(lambda x: list(set(x)))

###
all_defined_TA=pd.merge(all_defined_TA,df_remain_stores_Iter1_40,on="TA",how="left")

for col in ['store_id','All_P&S_zips','All_P&S_zips_40_miles']:
    all_defined_TA[col]=all_defined_TA[col].fillna("[]")
    all_defined_TA[col]=all_defined_TA[col].astype(str)
    all_defined_TA[col]=all_defined_TA[col].apply(lambda x: eval(x))
    
all_defined_TA['store_list']=all_defined_TA[['store_list','store_id']].sum(axis=1)
all_defined_TA['zip_cd_40']=all_defined_TA[['zip_cd_40','All_P&S_zips_40_miles']].sum(axis=1)
all_defined_TA['zip_cd_all']=all_defined_TA[['zip_cd_all','All_P&S_zips']].sum(axis=1)

all_defined_TA['zip_cd_40']=all_defined_TA['zip_cd_40'].apply(lambda x: list(set(x)))
all_defined_TA['zip_cd_all']=all_defined_TA['zip_cd_all'].apply(lambda x: list(set(x)))
all_defined_TA=all_defined_TA[['TA','store_list','zip_cd_40','zip_cd_all']]

df_remain_stores=df_remain_stores[~df_remain_stores['store_id'].isin(all_defined_TA['store_list'].sum())]

df_remain_stores['Max_overlap_TA']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[0])
df_remain_stores['Overlap_zips_with_TA_40']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[1])
df_remain_stores['Overlap_zips_count']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[2])
df_remain_stores['Overlap_zips_Pctg']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[3])
df_remain_stores.shape

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


(14, 41)

In [132]:
# Iter 2 - >=30% 

df_remain_stores_Iter1_40=df_remain_stores[df_remain_stores['Overlap_zips_Pctg']>=0.3]
df_remain_stores_Iter1_40=df_remain_stores_Iter1_40[['store_id','Max_overlap_TA','All_P&S_zips','All_P&S_zips_40_miles']]
df_remain_stores_Iter1_40=df_remain_stores_Iter1_40.rename(columns={"Max_overlap_TA":"TA"})
df_remain_stores_Iter1_40['store_id']=df_remain_stores_Iter1_40['store_id'].apply(lambda x: [x])
df_remain_stores_Iter1_40=df_remain_stores_Iter1_40.groupby("TA")['store_id','All_P&S_zips','All_P&S_zips_40_miles'].sum().reset_index()
for col in ['store_id','All_P&S_zips','All_P&S_zips_40_miles']:
    df_remain_stores_Iter1_40[col]=df_remain_stores_Iter1_40[col].apply(lambda x: list(set(x)))

###
all_defined_TA=pd.merge(all_defined_TA,df_remain_stores_Iter1_40,on="TA",how="left")

for col in ['store_id','All_P&S_zips','All_P&S_zips_40_miles']:
    all_defined_TA[col]=all_defined_TA[col].fillna("[]")
    all_defined_TA[col]=all_defined_TA[col].astype(str)
    all_defined_TA[col]=all_defined_TA[col].apply(lambda x: eval(x))
    
all_defined_TA['store_list']=all_defined_TA[['store_list','store_id']].sum(axis=1)
all_defined_TA['zip_cd_40']=all_defined_TA[['zip_cd_40','All_P&S_zips_40_miles']].sum(axis=1)
all_defined_TA['zip_cd_all']=all_defined_TA[['zip_cd_all','All_P&S_zips']].sum(axis=1)

all_defined_TA['zip_cd_40']=all_defined_TA['zip_cd_40'].apply(lambda x: list(set(x)))
all_defined_TA['zip_cd_all']=all_defined_TA['zip_cd_all'].apply(lambda x: list(set(x)))
all_defined_TA=all_defined_TA[['TA','store_list','zip_cd_40','zip_cd_all']]

df_remain_stores=df_remain_stores[~df_remain_stores['store_id'].isin(all_defined_TA['store_list'].sum())]

df_remain_stores['Max_overlap_TA']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[0])
df_remain_stores['Overlap_zips_with_TA_40']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[1])
df_remain_stores['Overlap_zips_count']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[2])
df_remain_stores['Overlap_zips_Pctg']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[3])
df_remain_stores.shape

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


(9, 41)

In [133]:
# Iter 3 - >=20% 

df_remain_stores_Iter1_40=df_remain_stores[df_remain_stores['Overlap_zips_Pctg']>=0.2]
df_remain_stores_Iter1_40=df_remain_stores_Iter1_40[['store_id','Max_overlap_TA','All_P&S_zips','All_P&S_zips_40_miles']]
df_remain_stores_Iter1_40=df_remain_stores_Iter1_40.rename(columns={"Max_overlap_TA":"TA"})
df_remain_stores_Iter1_40['store_id']=df_remain_stores_Iter1_40['store_id'].apply(lambda x: [x])
df_remain_stores_Iter1_40=df_remain_stores_Iter1_40.groupby("TA")['store_id','All_P&S_zips','All_P&S_zips_40_miles'].sum().reset_index()
for col in ['store_id','All_P&S_zips','All_P&S_zips_40_miles']:
    df_remain_stores_Iter1_40[col]=df_remain_stores_Iter1_40[col].apply(lambda x: list(set(x)))

###
all_defined_TA=pd.merge(all_defined_TA,df_remain_stores_Iter1_40,on="TA",how="left")

for col in ['store_id','All_P&S_zips','All_P&S_zips_40_miles']:
    all_defined_TA[col]=all_defined_TA[col].fillna("[]")
    all_defined_TA[col]=all_defined_TA[col].astype(str)
    all_defined_TA[col]=all_defined_TA[col].apply(lambda x: eval(x))
    
all_defined_TA['store_list']=all_defined_TA[['store_list','store_id']].sum(axis=1)
all_defined_TA['zip_cd_40']=all_defined_TA[['zip_cd_40','All_P&S_zips_40_miles']].sum(axis=1)
all_defined_TA['zip_cd_all']=all_defined_TA[['zip_cd_all','All_P&S_zips']].sum(axis=1)

all_defined_TA['zip_cd_40']=all_defined_TA['zip_cd_40'].apply(lambda x: list(set(x)))
all_defined_TA['zip_cd_all']=all_defined_TA['zip_cd_all'].apply(lambda x: list(set(x)))
all_defined_TA=all_defined_TA[['TA','store_list','zip_cd_40','zip_cd_all']]

df_remain_stores=df_remain_stores[~df_remain_stores['store_id'].isin(all_defined_TA['store_list'].sum())]

df_remain_stores['Max_overlap_TA']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[0])
df_remain_stores['Overlap_zips_with_TA_40']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[1])
df_remain_stores['Overlap_zips_count']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[2])
df_remain_stores['Overlap_zips_Pctg']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[3])
df_remain_stores.shape

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


(3, 41)

In [134]:
# Looks like the while loop is same as above

while df_remain_stores[df_remain_stores['Overlap_zips_Pctg']>0.2].shape[0]>0:
    df_remain_stores_Iter1_40=df_remain_stores[df_remain_stores['Overlap_zips_Pctg']>=0.2]
    df_remain_stores_Iter1_40=df_remain_stores_Iter1_40[['store_id','Max_overlap_TA','All_P&S_zips','All_P&S_zips_40_miles']]
    df_remain_stores_Iter1_40=df_remain_stores_Iter1_40.rename(columns={"Max_overlap_TA":"TA"})
    df_remain_stores_Iter1_40['store_id']=df_remain_stores_Iter1_40['store_id'].apply(lambda x: [x])
    df_remain_stores_Iter1_40=df_remain_stores_Iter1_40.groupby("TA")['store_id','All_P&S_zips','All_P&S_zips_40_miles'].sum().reset_index()
    for col in ['store_id','All_P&S_zips','All_P&S_zips_40_miles']:
        df_remain_stores_Iter1_40[col]=df_remain_stores_Iter1_40[col].apply(lambda x: list(set(x)))

    ###
    all_defined_TA=pd.merge(all_defined_TA,df_remain_stores_Iter1_40,on="TA",how="left")

    for col in ['store_id','All_P&S_zips','All_P&S_zips_40_miles']:
        all_defined_TA[col]=all_defined_TA[col].fillna("[]")
        all_defined_TA[col]=all_defined_TA[col].astype(str)
        all_defined_TA[col]=all_defined_TA[col].apply(lambda x: eval(x))

    all_defined_TA['store_list']=all_defined_TA[['store_list','store_id']].sum(axis=1)
    all_defined_TA['zip_cd_40']=all_defined_TA[['zip_cd_40','All_P&S_zips_40_miles']].sum(axis=1)
    all_defined_TA['zip_cd_all']=all_defined_TA[['zip_cd_all','All_P&S_zips']].sum(axis=1)

    all_defined_TA['zip_cd_40']=all_defined_TA['zip_cd_40'].apply(lambda x: list(set(x)))
    all_defined_TA['zip_cd_all']=all_defined_TA['zip_cd_all'].apply(lambda x: list(set(x)))
    all_defined_TA=all_defined_TA[['TA','store_list','zip_cd_40','zip_cd_all']]

    df_remain_stores=df_remain_stores[~df_remain_stores['store_id'].isin(all_defined_TA['store_list'].sum())]

    df_remain_stores['Max_overlap_TA']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[0])
    df_remain_stores['Overlap_zips_with_TA_40']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[1])
    df_remain_stores['Overlap_zips_count']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[2])
    df_remain_stores['Overlap_zips_Pctg']=df_remain_stores['All_P&S_zips_40_miles'].apply(lambda x: find_max_covered_TA(x)[3])
df_remain_stores.shape

(3, 41)

In [135]:
all_defined_TA['store_count']=all_defined_TA['store_list'].apply(lambda x: len(x))
all_defined_TA['store_count'].sum()

564

In [136]:
writer=pd.ExcelWriter(output_folder+"GY_remaining_stores_and_defined_2_TA_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_remain_stores.to_excel(writer,"df_remain_stores",index=False)
all_defined_TA.to_excel(writer,"all_defined_TA",index=False)
writer.save()


In [137]:
df_remain_stores_single_TA=df_remain_stores[['store_id','All_P&S_zips','All_P&S_zips_40_miles']]
df_remain_stores_single_TA['TA']=[all_defined_TA['TA'].max()+x+1 for x in range(len(df_remain_stores_single_TA))]
df_remain_stores_single_TA=df_remain_stores_single_TA.rename(columns={"store_id":"store_list"})
df_remain_stores_single_TA=df_remain_stores_single_TA.rename(columns={"All_P&S_zips_40_miles":"zip_cd_40"})
df_remain_stores_single_TA=df_remain_stores_single_TA.rename(columns={"All_P&S_zips":"zip_cd_all"})
df_remain_stores_single_TA['store_list']=df_remain_stores_single_TA['store_list'].apply(lambda x: [x])
df_remain_stores_single_TA.head(2)

for col in df_remain_stores_single_TA.columns.tolist():
    print(col,df_remain_stores_single_TA[col].apply(lambda x: type(x)).unique())
    
for col in all_defined_TA.columns.tolist():
    print(col,all_defined_TA[col].apply(lambda x: type(x)).unique())
    
all_defined_TA=all_defined_TA.append(df_remain_stores_single_TA)

all_defined_TA['store_count']=all_defined_TA['store_list'].apply(lambda x: len(x))

store_list [<class 'list'>]
zip_cd_all [<class 'list'>]
zip_cd_40 [<class 'list'>]
TA [<class 'int'>]
TA [<class 'int'>]
store_list [<class 'list'>]
zip_cd_40 [<class 'list'>]
zip_cd_all [<class 'list'>]
store_count [<class 'int'>]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [138]:
all_defined_TA['store_count']=all_defined_TA['store_list'].apply(lambda x: len(x))

all_defined_TA['temp_1st_store']=all_defined_TA['store_list'].apply(lambda x: "single_"+str(x[0]))
all_defined_TA['temp_TA_str']=all_defined_TA['TA'].apply(lambda x: "multiple_"+str(x))

all_defined_TA['ta_name']=np.where(all_defined_TA['store_count']==1,all_defined_TA['temp_1st_store'],all_defined_TA['temp_TA_str'])

del all_defined_TA['temp_1st_store']
del all_defined_TA['temp_TA_str']

In [139]:
summary_store_count_TA=all_defined_TA.groupby("store_count")['TA'].count().to_frame().reset_index().sort_values("store_count",ascending=False)

In [140]:
# Start writing out final result

df_output_by_store=pd.DataFrame(columns={"store_id","TA","ta_name"})

for ind,row in all_defined_TA.iterrows():
    store_list_in_TA=row['store_list']
    TA_num=row['TA']
    TA_name=row['ta_name']
    df=pd.DataFrame({"store_id":store_list_in_TA,"TA":[TA_num]*len(store_list_in_TA),"ta_name":[TA_name]*len(store_list_in_TA)},index=range(len(store_list_in_TA)))
    df_output_by_store=df_output_by_store.append(df)
df_output_by_store=df_output_by_store.sort_values(["TA","store_id"])
df_output_by_store.head(3)

,TA,ta_name,store_id
3,1,multiple_1,1290
2,1,multiple_1,4861
0,1,multiple_1,8724


In [141]:
# Defined TAs and associated zips

df_previous_zip_by_store_copy=df_sales_by_zip[df_sales_by_zip['store_id'].isin(df_output_by_store['store_id'].tolist())]
df_previous_zip_by_store_copy=df_previous_zip_by_store_copy[['store_id','zip_5','trans_label']]
df_previous_zip_by_store_copy=df_previous_zip_by_store_copy[df_previous_zip_by_store_copy['trans_label'].isin(["P","S"])]
df_previous_zip_by_store_copy=df_previous_zip_by_store_copy.rename(columns={"zip_5":"zip_cd"})

print(df_previous_zip_by_store_copy.shape)

df_trans_zips=df_previous_zip_by_store_copy.groupby(["store_id","trans_label"])['zip_cd'].apply(list).to_frame().reset_index()

df_trans_zips=df_trans_zips.pivot(index="store_id",columns="trans_label",values="zip_cd").reset_index()
df_trans_zips=df_trans_zips.rename(columns={"P":"all_trans_P_zips","S":"all_trans_S_zips"})
# df_10_zips=df_new_store_zips.groupby("store_id")['zips_in_10'].apply(list).to_frame().reset_index()
# df_output_by_store_zips=df_trans_zips.append(df_10_zips)
df_output_by_store_zips=df_trans_zips.copy()

(398962, 3)


In [142]:
# Merge in

df_output_by_store=pd.merge(df_output_by_store,store_list,on="store_id",how="left")
df_output_by_store=pd.merge(df_output_by_store,DMA_nielsen,on="zip_cd",how="left")
df_output_by_store=pd.merge(df_output_by_store,df_output_by_store_zips,on="store_id",how="left")


In [143]:
# Fullfil nan with "[]" and eval into list
df_output_by_store['all_trans_P_zips']=df_output_by_store['all_trans_P_zips'].fillna("[]")
df_output_by_store['all_trans_S_zips']=df_output_by_store['all_trans_S_zips'].fillna("[]")
# df_output_by_store['zips_in_10']=df_output_by_store['zips_in_10'].fillna("[]")

df_output_by_store['all_trans_P_zips']=df_output_by_store['all_trans_P_zips'].astype(str)
df_output_by_store['all_trans_S_zips']=df_output_by_store['all_trans_S_zips'].astype(str)
# df_output_by_store['zips_in_10']=df_output_by_store['zips_in_10'].astype(str)

df_output_by_store['all_trans_P_zips']=df_output_by_store['all_trans_P_zips'].apply(lambda x: eval(x))
df_output_by_store['all_trans_S_zips']=df_output_by_store['all_trans_S_zips'].apply(lambda x: eval(x))
# df_output_by_store['zips_in_10']=df_output_by_store['zips_in_10'].apply(lambda x: eval(x))

In [144]:
# All the same above, just my preference to start from raw to see if result in the same, work somehow to QC
# In addition, to see as TA now, any TA can be combined together??
# Limit 40 miles zips to the associated stores

df_output_by_store_copy=df_output_by_store.copy()
df_output_by_store_copy=df_output_by_store_copy.reset_index()
del df_output_by_store_copy['index']

location_lat_long_dict=df_output_by_store_copy[['store_id','latitude','longitude']].drop_duplicates()
location_lat_long_dict['store_center']=location_lat_long_dict[['latitude','longitude']].values.tolist()
location_lat_long_dict=location_lat_long_dict.set_index("store_id")['store_center']


df_output_by_store_1=pd.DataFrame()
df_output_by_store_copy['trans_P_zips_40_within_TA']=np.nan
df_output_by_store_copy['trans_S_zips_40_within_TA']=np.nan

for TA,df_group in df_output_by_store_copy.groupby("TA"):
    store_list=df_group['store_id'].tolist()
    df_group=df_group.reset_index()
    del df_group['index']
    
    for ind,row in df_group.iterrows():
        all_P_zips_list=row['all_trans_P_zips']
        all_S_zips_list=row['all_trans_S_zips']
        P_zips_list_40_in_TA=[]
        S_zips_list_40_in_TA=[]
        
        for zip_P in all_P_zips_list:
            if zip_P in zip_centers.keys():
                for store in store_list:
                    store_center=location_lat_long_dict[store]
                    dist=haversine(store_center, zip_centers[zip_P],unit="mi")
                    if dist<=num_final_inclusion_dist:
                        P_zips_list_40_in_TA=P_zips_list_40_in_TA+[zip_P]
                
        for zip_S in all_S_zips_list:
            if zip_S in zip_centers.keys():
                for store in store_list:
                    store_center=location_lat_long_dict[store]
                    dist=haversine(store_center, zip_centers[zip_S],unit="mi")
                    if dist<=num_final_inclusion_dist:
                        S_zips_list_40_in_TA=S_zips_list_40_in_TA+[zip_S]
                    
        P_zips_list_40_in_TA=list(set(P_zips_list_40_in_TA))
        S_zips_list_40_in_TA=list(set(S_zips_list_40_in_TA))
        
        df_group.loc[ind,'trans_P_zips_40_within_TA']=str(P_zips_list_40_in_TA)
        df_group.loc[ind,'trans_S_zips_40_within_TA']=str(S_zips_list_40_in_TA)
        
    df_output_by_store_1=df_output_by_store_1.append(df_group)
        
df_output_by_store_1['trans_P_zips_40_within_TA']=df_output_by_store_1['trans_P_zips_40_within_TA'].apply(lambda x: eval(x))
df_output_by_store_1['trans_S_zips_40_within_TA']=df_output_by_store_1['trans_S_zips_40_within_TA'].apply(lambda x: eval(x))

df_output_by_store=df_output_by_store_1.copy()

In [145]:
df_output_by_store.head(5)

,TA,ta_name,store_id,address_1,zip_cd,city,state,latitude,longitude,DMA,CTY,all_trans_P_zips,all_trans_S_zips,trans_P_zips_40_within_TA,trans_S_zips_40_within_TA
0,1,multiple_1,1290,15498 EAST 14TH ST,95123,San Leandro,CA,37.250549,-121.867305,{SAN FRANCISCO-OAK-SAN JOSE},{SANTA CLARA},"[95123, 94578, 94010, 94577, 95035, 94116, 941...","[94043, 94121, 95030, 94070, 78613, 95136, 951...","[94568, 94577, 94538, 94536, 94539, 95124, 951...","[94403, 95148, 94402, 95110, 95127, 95136, 945..."
1,1,multiple_1,4861,15751 LOS GATOS BLVD,95032,LOS GATOS,CA,37.239935,-121.961685,{SAN FRANCISCO-OAK-SAN JOSE},{SANTA CLARA},"[95032, 95030, 95008]","[95124, 95033, 95070, 95125, 95118, 95120, 950...","[95030, 95008, 95032]","[95003, 95148, 94402, 94035, 95009, 95065, 945..."
2,1,multiple_1,8724,3800 CAPITOLA ROAD,95010,CAPITOLA,CA,36.973444,-121.967701,{MONTEREY-SALINAS},{SANTA CRUZ},[95062],"[95010, 95003, 95073, 95060, 95076, 95001, 950...",[95062],"[95003, 95039, 95148, 94402, 94035, 94613, 950..."
3,1,multiple_1,9246,995 BLOSSOM HILL ROAD,95123,SAN JOSE,CA,37.250549,-121.867305,{SAN FRANCISCO-OAK-SAN JOSE},{SANTA CLARA},"[95123, 95136, 95118]","[95120, 95124, 95125, 95111, 95037, 95138, 951...","[95136, 95118, 95123]","[95003, 95148, 94402, 94035, 95009, 94583, 951..."
0,2,multiple_2,2121,100 PEACHTREE INDUSTRIAL BLVD,30518,SUGAR HILL,GA,34.074493,-84.062975,{ATLANTA},"{GWINNETT, HALL}",[30024],"[30518, 30519, 30542, 30097, 30096, 30043, 305...",[30024],"[30253, 30311, 30350, 30107, 30101, 30308, 303..."


In [146]:
df_output_by_store=df_output_by_store.reset_index()
del df_output_by_store['index']

df_output_by_store_zip_long=pd.DataFrame(columns=["TA","ta_name","store_id",'zip_cd','zip_type'])

for ind,row in df_output_by_store.iterrows():
    TA=row['TA']
    ta_name=row['ta_name']
    store_id=row['store_id']
    df=pd.DataFrame()
    
    all_P_zips=row['trans_P_zips_40_within_TA']
    all_S_zips=row['trans_S_zips_40_within_TA']
    # all_10_zips=row['zips_in_10']
    
    df_P=pd.DataFrame({"store_id":[store_id]*len(all_P_zips),'zip_cd':all_P_zips})
    df_P['zip_type']="trans_P"
    
    df_S=pd.DataFrame({"store_id":[store_id]*len(all_S_zips),'zip_cd':all_S_zips})
    df_S['zip_type']="trans_S"
    
    # df_10=pd.DataFrame({"store_id":[store_id]*len(all_10_zips),'zip_cd':all_10_zips})
    # df_10['zip_type']="zips_10"
    
    df=df_P.append(df_S)
    df['TA']=TA
    df['ta_name']=ta_name
    df_output_by_store_zip_long=df_output_by_store_zip_long.append(df)

In [147]:
df_output_by_store_zip_long.head(2)

,TA,ta_name,store_id,zip_cd,zip_type
0,1,multiple_1,1290,94568,trans_P
1,1,multiple_1,1290,94577,trans_P


In [148]:
df_output_by_TA=df_output_by_store_zip_long.sort_values(["TA",'ta_name','zip_type','zip_cd'])
df_output_by_TA=df_output_by_TA.drop_duplicates(["TA",'ta_name','zip_cd'])

df_output_by_TA=df_output_by_TA.groupby(["TA","ta_name","zip_type"])['zip_cd'].apply(list).to_frame().reset_index()
df_output_by_TA=df_output_by_TA.pivot(index="TA",columns="zip_type",values="zip_cd").reset_index()


ta_name=df_output_by_store_zip_long[['TA','ta_name']].drop_duplicates()
df_output_by_TA=pd.merge(df_output_by_TA,ta_name,on="TA",how="left")

ta_store_list=df_output_by_store.groupby("TA")['store_id'].apply(list).to_frame().reset_index()

df_output_by_TA=pd.merge(df_output_by_TA,ta_store_list,on='TA',how="left")

df_output_by_TA=df_output_by_TA.rename(columns={"store_id":"store_list","trans_P":"trans_P_zips","trans_S":"trans_S_zips"})
df_output_by_TA=df_output_by_TA[['TA','ta_name','store_list','trans_P_zips','trans_S_zips']]
df_output_by_TA['store_count']=df_output_by_TA['store_list'].apply(lambda x: len(x))

df_output_by_TA.head(2)

,TA,ta_name,store_list,trans_P_zips,trans_S_zips,store_count
0,1,multiple_1,"[1290, 4861, 8724, 9246]","[94010, 94025, 94044, 94066, 94080, 94085, 940...","[93901, 93902, 93905, 93906, 93907, 93908, 939...",4
1,2,multiple_2,"[2121, 2124, 2126, 2128, 2130, 2136, 2141, 3610]","[30004, 30008, 30017, 30019, 30024, 30030, 300...","[30002, 30005, 30009, 30010, 30011, 30012, 300...",8


In [149]:
# Merge multiple TA with 30% overlap + 
df_output_by_TA['trans_P_zips']=df_output_by_TA['trans_P_zips'].fillna("[]").astype(str).apply(lambda x: eval(x))
df_output_by_TA['trans_S_zips']=df_output_by_TA['trans_S_zips'].fillna("[]").astype(str).apply(lambda x: eval(x))
# df_output_by_TA['distance_10_zips']=df_output_by_TA['distance_10_zips'].fillna("[]").astype(str).apply(lambda x: eval(x))

df_output_by_TA=df_output_by_TA.reset_index()
del df_output_by_TA['index']
df_output_by_TA['all_zips']=df_output_by_TA['trans_P_zips']+df_output_by_TA['trans_S_zips'] #+df_output_by_TA['distance_10_zips']


In [150]:
df_qc_overlap_1=df_output_by_TA[['TA','ta_name','all_zips']].rename(columns={"TA":"TA_1","all_zips":"all_zips_1"})
df_qc_overlap_1=df_qc_overlap_1[df_qc_overlap_1['ta_name'].apply(lambda x: x.split("_")[0]=="multiple")]
df_qc_overlap_1['temp']=1
del df_qc_overlap_1['ta_name']

df_qc_overlap_2=df_output_by_TA[['TA','ta_name','all_zips']].rename(columns={"TA":"TA_2","all_zips":"all_zips_2"})
df_qc_overlap_2=df_qc_overlap_2[df_qc_overlap_2['ta_name'].apply(lambda x: x.split("_")[0]=="multiple")]
df_qc_overlap_2['temp']=1
del df_qc_overlap_2['ta_name']

df_qc_overlap=pd.merge(df_qc_overlap_1,df_qc_overlap_2,on="temp",how="outer")
df_qc_overlap=df_qc_overlap[df_qc_overlap['TA_1']!=df_qc_overlap['TA_2']]
del df_qc_overlap['temp']

In [151]:
def intsection_pctg_qc_TA(list_1,list_2):
    intersection_list=list(set(list_1).intersection(set(list_2)))
    intersection_pctg=len(intersection_list)/len(set(list_1+list_2))
    return intersection_pctg

df_qc_overlap['intersecion_pctg']=df_qc_overlap.apply(lambda x: intsection_pctg_qc_TA(x['all_zips_1'],x['all_zips_2']),axis=1)

In [152]:
df_overlap_high_TA=df_qc_overlap[df_qc_overlap['intersecion_pctg']>0.3]
df_overlap_high_TA['pairs']=df_overlap_high_TA[['TA_1','TA_2']].values.tolist()
df_overlap_high_TA['pairs']=df_overlap_high_TA['pairs'].apply(lambda x: str(sorted(x)))
df_overlap_high_TA=df_overlap_high_TA[['TA_1','TA_2','pairs']].drop_duplicates()

df_overlap_high_TA_all_pairs=df_overlap_high_TA['pairs'].unique().tolist()
df_overlap_high_TA_all_pairs=[eval(x) for x in df_overlap_high_TA_all_pairs] # actually a list

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [153]:
df_overlap_high_TA.head(3)

,TA_1,TA_2,pairs
3,1,4,"[1, 4]"
115,1,116,"[1, 116]"
140,2,22,"[2, 22]"


In [154]:
merge_M_M_TA_dict={}
for pair_list in df_overlap_high_TA_all_pairs:
    merge_M_M_TA_dict.update({pair_list[1]:pair_list[0]}) 
    
merge_M_M_TA_dict

{4: 1,
 116: 4,
 22: 2,
 80: 22,
 96: 5,
 72: 6,
 12: 7,
 19: 8,
 76: 11,
 64: 13,
 65: 64,
 71: 65,
 90: 71,
 99: 65,
 66: 16,
 70: 66,
 81: 70,
 77: 17,
 98: 77,
 40: 21,
 75: 21,
 28: 23,
 43: 36,
 94: 44,
 42: 41,
 92: 42,
 113: 51,
 82: 54,
 85: 82,
 109: 55,
 106: 58,
 101: 67,
 79: 74,
 112: 100}

In [155]:
del df_output_by_store['ta_name']
df_output_by_store_1=df_output_by_store[df_output_by_store['TA'].isin(merge_M_M_TA_dict.keys())]
df_output_by_store_2=df_output_by_store[~df_output_by_store['TA'].isin(merge_M_M_TA_dict.keys())]

df_output_by_store_1['TA']=df_output_by_store_1['TA'].apply(lambda x: merge_M_M_TA_dict[x])
df_output_by_store=df_output_by_store_1.append(df_output_by_store_2)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [156]:
# Naming TA
df_ta_name=df_output_by_store.groupby("TA")['store_id'].apply(list).to_frame().reset_index()
df_ta_name['store_count']=df_ta_name['store_id'].apply(lambda x: len(x))
df_ta_name=df_ta_name.sort_values("store_count",ascending=False)
df_ta_name['TA_num']=[str(x+1) for x in range(len(df_ta_name))]
df_ta_name['store_id_0_temp']=df_ta_name['store_id'].apply(lambda x: x[0])
df_ta_name['store_id_0_temp']="single_"+df_ta_name['store_id_0_temp']
df_ta_name['multiple_temp']="multiple_"+df_ta_name['TA_num']

df_ta_name['ta_name']=np.where(df_ta_name['store_count']==1,df_ta_name['store_id_0_temp'],df_ta_name['multiple_temp'])

df_ta_name=df_ta_name[['TA','TA_num','ta_name']]

In [157]:
# Generate views by store, TA and zip

df_output_by_store=pd.merge(df_output_by_store,df_ta_name,on="TA",how="left")
df_output_by_store=df_output_by_store[['TA_num','ta_name']+[x for x in df_output_by_store.columns.tolist() if x not in ['TA_num','ta_name']]]

del df_output_by_store['TA']
df_output_by_store['TA_num']=df_output_by_store['TA_num'].astype(int)
df_output_by_store=df_output_by_store.sort_values("TA_num")

In [158]:
df_output_by_store=df_output_by_store.reset_index()
del df_output_by_store['index']

df_output_by_store_zip_long=pd.DataFrame(columns=["TA_num","ta_name","store_id",'zip_cd','zip_type'])

for ind,row in df_output_by_store.iterrows():
    TA=row['TA_num']
    ta_name=row['ta_name']
    store_id=row['store_id']
    df=pd.DataFrame()
    
    all_P_zips=row['trans_P_zips_40_within_TA']
    all_S_zips=row['trans_S_zips_40_within_TA']
    # all_10_zips=row['zips_in_10']
    
    df_P=pd.DataFrame({"store_id":[store_id]*len(all_P_zips),'zip_cd':all_P_zips})
    df_P['zip_type']="trans_P"
    
    df_S=pd.DataFrame({"store_id":[store_id]*len(all_S_zips),'zip_cd':all_S_zips})
    df_S['zip_type']="trans_S"
    
    # df_10=pd.DataFrame({"store_id":[store_id]*len(all_10_zips),'zip_cd':all_10_zips})
    # df_10['zip_type']="zips_10"
    
    df=df_P.append(df_S)#.append(df_10)
    df['TA_num']=TA
    df['ta_name']=ta_name
    df_output_by_store_zip_long=df_output_by_store_zip_long.append(df)

In [159]:
df_output_by_TA=df_output_by_store_zip_long.sort_values(["TA_num",'ta_name','zip_type','zip_cd'])
df_output_by_TA=df_output_by_TA.drop_duplicates(["TA_num",'ta_name','zip_cd'])

df_output_by_TA=df_output_by_TA.groupby(["TA_num","ta_name","zip_type"])['zip_cd'].apply(list).to_frame().reset_index()
df_output_by_TA=df_output_by_TA.pivot(index="TA_num",columns="zip_type",values="zip_cd").reset_index()


ta_name=df_output_by_store_zip_long[['TA_num','ta_name']].drop_duplicates()
df_output_by_TA=pd.merge(df_output_by_TA,ta_name,on="TA_num",how="left")

ta_store_list=df_output_by_store.groupby("TA_num")['store_id'].apply(list).to_frame().reset_index()

df_output_by_TA=pd.merge(df_output_by_TA,ta_store_list,on='TA_num',how="left")

df_output_by_TA=df_output_by_TA.rename(columns={"store_id":"store_list","trans_P":"trans_P_zips","trans_S":"trans_S_zips"})
df_output_by_TA=df_output_by_TA[['TA_num','ta_name','store_list','trans_P_zips','trans_S_zips']]
df_output_by_TA['store_count']=df_output_by_TA['store_list'].apply(lambda x: len(x))


df_output_by_TA.head(2)


,TA_num,ta_name,store_list,trans_P_zips,trans_S_zips,store_count
0,1,multiple_1,"[1270, 6143, 6133, 6125, 6120, 1268, 1427, 128...","[60004, 60005, 60007, 60008, 60010, 60015, 600...","[46301, 46303, 46304, 46307, 46310, 46311, 463...",23
1,2,multiple_2,"[6651, 6675, 6663, 6661, 6647, 2544, 6623, 662...","[33018, 33019, 33020, 33022, 33023, 33024, 330...","[33002, 33004, 33008, 33009, 33010, 33012, 330...",19


In [160]:
df_output_unique_zips=df_output_by_store_zip_long[['zip_type','zip_cd']].drop_duplicates()
df_output_unique_zips=df_output_unique_zips.sort_values("zip_type")
df_output_unique_zips=df_output_unique_zips.drop_duplicates("zip_cd")
df_output_unique_zips.shape

(17071, 2)

In [161]:
location_lat_long=df_store_included[['store_num','latitude','longitude']].rename(columns={"store_num":"store_id"})
df_output_by_store_zip_long=pd.merge(df_output_by_store_zip_long,location_lat_long,on="store_id",how="left")


In [162]:
list_store_missing=[x for x in df_store_included['store_num'].tolist() if x not in df_output_by_store['store_id'].tolist()]
list_store_missing

[]

In [163]:
df_output_by_TA['P_zip_counts']=df_output_by_TA['trans_P_zips'].apply(len)
df_output_by_TA['S_zip_counts']=df_output_by_TA['trans_S_zips'].apply(len)

df_output_by_store['all_P_zip_counts']=df_output_by_store['all_trans_P_zips'].apply(len)
df_output_by_store['all_S_zip_counts']=df_output_by_store['all_trans_S_zips'].apply(len)

df_output_by_store['mile40_P_zip_counts']=df_output_by_store['trans_P_zips_40_within_TA'].apply(len)
df_output_by_store['mile40_S_zip_counts']=df_output_by_store['trans_S_zips_40_within_TA'].apply(len)


In [164]:
list_1_ta_view=df_output_by_TA['store_list'].sum()
list_2_store_view=df_output_by_store['store_id'].unique().tolist()
list_3_whole=df_store_included['store_num'].tolist()


In [165]:
# Double check cover all
set(list_3_whole)-set(list_1_ta_view)

set()

In [166]:
df_output_by_store[df_output_by_store['store_id']=='2393']

,TA_num,ta_name,store_id,address_1,zip_cd,city,state,latitude,longitude,DMA,CTY,all_trans_P_zips,all_trans_S_zips,trans_P_zips_40_within_TA,trans_S_zips_40_within_TA,all_P_zip_counts,all_S_zip_counts,mile40_P_zip_counts,mile40_S_zip_counts
309,27,multiple_27,2393,114 ROLLING HILLS CIRCLE,29640,EASLEY,SC,34.862904,-82.550289,{GREENVLL-SPART-ASHEVLL-AND},"{GREENVILLE, PICKENS}",[29642],"[29640, 29671, 29657, 29673, 29611, 29630, 292...",[29642],"[29672, 29627, 28768, 29320, 29641, 28736, 296...",1,963,1,120


# Additional Information

In [167]:
# Append more information by store, query from the DB for the columns

query_string='''
SELECT t1.installer_store_num, t1.store_num, t1.company_owned_store_ind, t1.zip_cd  from mdb.vw__installer_master as t1
left join mdb_cdl.store t2 on t1.store_num=t2.store_num
where (open_store_flag='Y' or open_store_flag is null and company_owned_store_ind="Y") or (company_owned_store_ind="N" and t1.close_date is null);
'''

print("query: \n%s"%query_string)
df_all_open_stores=client.query(query_string).result().to_dataframe()
print("df_all_open_stores.shape",df_all_open_stores.shape)
df_all_open_stores['store_num']=df_all_open_stores['store_num'].astype(int).astype(str)
df_all_open_stores['zip_cd']=df_all_open_stores['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
print(df_all_open_stores.shape,df_all_open_stores['installer_store_num'].nunique())

query: 

SELECT t1.installer_store_num, t1.store_num, t1.company_owned_store_ind, t1.zip_cd  from mdb.vw__installer_master as t1
left join mdb_cdl.store t2 on t1.store_num=t2.store_num
where (open_store_flag='Y' or open_store_flag is null and company_owned_store_ind="Y") or (company_owned_store_ind="N" and t1.close_date is null);



/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


df_all_open_stores.shape (4639, 4)
(4639, 4) 4639


In [168]:
df_store_count_by_zip=df_all_open_stores.groupby(["zip_cd","company_owned_store_ind"])['installer_store_num'].count().to_frame().reset_index()
df_store_count_by_zip=df_store_count_by_zip.pivot_table(index='zip_cd',columns="company_owned_store_ind",values='installer_store_num').reset_index()
df_store_count_by_zip=df_store_count_by_zip.fillna(0)
df_store_count_by_zip=df_store_count_by_zip.rename(columns={"Y":"coor_store_counts","N":"non_coor_store_counts"})
df_store_count_by_zip['coor_store_counts']=df_store_count_by_zip['coor_store_counts'].astype(int)
df_store_count_by_zip['non_coor_store_counts']=df_store_count_by_zip['non_coor_store_counts'].astype(int)

df_store_count_by_zip.head(3)

company_owned_store_ind,zip_cd,non_coor_store_counts,coor_store_counts
0,01056,2,0
1,01089,1,0
2,01201,1,0


In [169]:
df_store_count_by_zip['coor_store_counts'].sum()

573

In [170]:
df_output_by_TA=df_output_by_TA.reset_index()
del df_output_by_TA['index']
df_output_by_TA.insert(3,'city_list',np.nan)
df_output_by_TA.insert(4,'state_list',np.nan)
df_output_by_TA['count_Coor_store_in_P']=np.nan
df_output_by_TA['count_Noncoor_store_in_P']=np.nan
df_output_by_TA['count_Coor_store_in_S']=np.nan
df_output_by_TA['count_Noncoor_store_in_S']=np.nan

for ind, row in df_output_by_TA.iterrows():
    list_stores=row['store_list']
    df_included_store_city=df_store_included[df_store_included['store_num'].isin(list_stores)]
    list_cities=list(set(df_included_store_city['city'].tolist()))
    list_states=list(set(df_included_store_city['state'].tolist()))
    
    list_P=row['trans_P_zips']
    count_coor_stores_P=df_store_count_by_zip[df_store_count_by_zip['zip_cd'].isin(list_P)]['coor_store_counts'].sum()
    count_noncoor_stores_P=df_store_count_by_zip[df_store_count_by_zip['zip_cd'].isin(list_P)]['non_coor_store_counts'].sum()
    
    list_S=row['trans_S_zips']
    count_coor_stores_S=df_store_count_by_zip[df_store_count_by_zip['zip_cd'].isin(list_S)]['coor_store_counts'].sum()
    count_noncoor_stores_S=df_store_count_by_zip[df_store_count_by_zip['zip_cd'].isin(list_S)]['non_coor_store_counts'].sum()
    
    df_output_by_TA.loc[ind,'city_list']=str(list_cities)
    df_output_by_TA.loc[ind,'state_list']=str(list_states)
    df_output_by_TA.loc[ind,'count_Coor_store_in_P']=str(count_coor_stores_P)
    df_output_by_TA.loc[ind,'count_Noncoor_store_in_P']=str(count_noncoor_stores_P)
    df_output_by_TA.loc[ind,'count_Coor_store_in_S']=str(count_coor_stores_S)
    df_output_by_TA.loc[ind,'count_Noncoor_store_in_S']=str(count_noncoor_stores_S)
df_output_by_TA.head(2)

,TA_num,ta_name,store_list,city_list,state_list,trans_P_zips,trans_S_zips,store_count,P_zip_counts,S_zip_counts,count_Coor_store_in_P,count_Noncoor_store_in_P,count_Coor_store_in_S,count_Noncoor_store_in_S
0,1,multiple_1,"[1270, 6143, 6133, 6125, 6120, 1268, 1427, 128...","['SKOKIE', 'MORTON GROVE', 'NORRIDGE', 'HANOVE...",['IL'],"[60004, 60005, 60007, 60008, 60010, 60015, 600...","[46301, 46303, 46304, 46307, 46310, 46311, 463...",23,86,491,19,29,6,108
1,2,multiple_2,"[6651, 6675, 6663, 6661, 6647, 2544, 6623, 662...","['SOUTHWEST RANCHES', 'CORAL GABLES', 'FORT LA...",['FL'],"[33018, 33019, 33020, 33022, 33023, 33024, 330...","[33002, 33004, 33008, 33009, 33010, 33012, 330...",19,63,239,16,20,6,44


In [171]:
df_output_by_TA['city_list']=df_output_by_TA['city_list'].apply(eval)
df_output_by_TA['state_list']=df_output_by_TA['state_list'].apply(eval)
df_output_by_TA['count_Coor_store_in_P']=df_output_by_TA['count_Coor_store_in_P'].apply(eval)
df_output_by_TA['count_Noncoor_store_in_P']=df_output_by_TA['count_Noncoor_store_in_P'].apply(eval)
df_output_by_TA['count_Coor_store_in_S']=df_output_by_TA['count_Coor_store_in_S'].apply(eval)
df_output_by_TA['count_Noncoor_store_in_S']=df_output_by_TA['count_Noncoor_store_in_S'].apply(eval)


In [172]:
if len(list_store_missing)==0:
    writer=pd.ExcelWriter(output_folder+"GY_final_TA_updated_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
    df_output_unique_zips.to_excel(writer,"unique_zips_full_footprint",index=False)
    df_output_by_TA.to_excel(writer,"view_by_TA",index=False)
    df_output_by_store.to_excel(writer,"view_by_store",index=False)
    df_output_by_store_zip_long.to_excel(writer,"view_for_Tableau",index=False)
    writer.save()
else:
    print("checking stores misssing: \n%s"%list_store_missing)


In [167]:
df_output_by_store[df_output_by_store['store_id']=='2393']

,TA_num,ta_name,store_id,address_1,zip_cd,city,state,latitude,longitude,DMA,CTY,all_trans_P_zips,all_trans_S_zips,trans_P_zips_40_within_TA,trans_S_zips_40_within_TA,all_P_zip_counts,all_S_zip_counts,mile40_P_zip_counts,mile40_S_zip_counts
309,27,multiple_27,2393,114 ROLLING HILLS CIRCLE,29640,EASLEY,SC,34.862904,-82.550289,{GREENVLL-SPART-ASHEVLL-AND},"{PICKENS, GREENVILLE}",[29642],"[29640, 29671, 29657, 29673, 29611, 29630, 292...",[29642],"[29617, 29672, 29639, 29306, 29656, 29688, 296...",1,963,1,120


In [ ]:
# Finished